# Data Analysis ETH Transactions

# Data - Accredited Investors

## get data from Google API

In [1]:
#set up google sheets API
from googleapiclient.discovery import build 
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'keys.json'

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1cK_i04kps5_R2dIvCe2Y-Yi3MxLj6QcMlI8Za8-gdAk'
service = build('sheets', 'v4', credentials=creds)
SAMPLE_RANGE_NAME = "SoW_approved!A1:E356"

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])
print(values[:5]) 

[['sow_estimated_net_worth_value', 'address', 'number', 'wallet_address', 'full_name'], ['30000', 'c.schreiner@me.com', '', '', 'Christian Paul Schreiner'], ['250000', 'elmotec@t-online.de', '', '', 'Bodo Jürgen Otto Cuny'], ['300', 'axel.nitsch@gmail.com', '', '', 'Axel Gunnar Nitsch'], ['140', 'ralph.mende@hotmail.de', '', '', 'Ralph Klaus Mende']]


## explore the data

In [2]:
import pandas as pd
import numpy as np

from tqdm import notebook
notebook.tqdm().pandas()

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
df = pd.DataFrame(values)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [5]:
df

,sow_estimated_net_worth_value,address,number,wallet_address,full_name
1,30000,c.schreiner@me.com,,,Christian Paul Schreiner
2,250000,elmotec@t-online.de,,,Bodo Jürgen Otto Cuny
3,300,axel.nitsch@gmail.com,,,Axel Gunnar Nitsch
4,140,ralph.mende@hotmail.de,,,Ralph Klaus Mende
5,100000,pmedam@gmail.com,,398Lx4SsoCHwULhewCwYJd6ii4FTV8sFig,Peter Josef Medam
...,...,...,...,...,...
350,350000,b_bordenave@live.com,,0x06f62B8B2eebdE73dd4F0E1fA9b1321D63fd5c3A,Benjamin Bordenave
351,100000,snakecha@ellipti.io,,,SEUNGHOON CHA
352,345000,ltmemories@gmail.com,,0xD5aBc758D753B68985B2Bb83F2b2F4030179ea20,John Gravagna
353,50000,seleznyov@gmail.com,,0x0CEAd33EB6AdF4860B6b8F66687f51567E111973,ANDRII SELEZNOV


## clean the data

In [6]:
#only keep observations with a wallet_address
df = df.drop(columns=['number']) #drop empty column
df_NA = df.replace('^\s*$', np.nan, regex=True) #encode empty space as NA
df = df_NA.dropna()
df

,sow_estimated_net_worth_value,address,wallet_address,full_name
5,100000,pmedam@gmail.com,398Lx4SsoCHwULhewCwYJd6ii4FTV8sFig,Peter Josef Medam
12,10000,henrik.gebbing@gmail.com,3Ef1iMk3eTmTsKxtunCu2pWYW3Dvq38Ba6,Henrik Gebbing
13,60,jan@finoa.io,3B2i7TiDypGcPxeNNS2w9TkrZ4xeAU3jJ9,Jan-Peter Christoph Kind
14,2,marius-gebbing@gmx.de,1CpKddUS9WNy5JQhjApvNzJuyAqX5b3wn7,Marius Gebbing
15,6000,mizusize@gmail.com,0x3296B13B1eE66D953d4449BD81581a4612ECc27F,AKIHIRO MIZUGUCHI
...,...,...,...,...
349,1000000,shahsaumil@gmail.com,0x0230c0F62a61D77c62C48B162F6dec2Be8DC4D20,SAUMIL G SHAH
350,350000,b_bordenave@live.com,0x06f62B8B2eebdE73dd4F0E1fA9b1321D63fd5c3A,Benjamin Bordenave
352,345000,ltmemories@gmail.com,0xD5aBc758D753B68985B2Bb83F2b2F4030179ea20,John Gravagna
353,50000,seleznyov@gmail.com,0x0CEAd33EB6AdF4860B6b8F66687f51567E111973,ANDRII SELEZNOV


In [7]:
#create a copy of the old df
df2 = df
df2

,sow_estimated_net_worth_value,address,wallet_address,full_name
5,100000,pmedam@gmail.com,398Lx4SsoCHwULhewCwYJd6ii4FTV8sFig,Peter Josef Medam
12,10000,henrik.gebbing@gmail.com,3Ef1iMk3eTmTsKxtunCu2pWYW3Dvq38Ba6,Henrik Gebbing
13,60,jan@finoa.io,3B2i7TiDypGcPxeNNS2w9TkrZ4xeAU3jJ9,Jan-Peter Christoph Kind
14,2,marius-gebbing@gmx.de,1CpKddUS9WNy5JQhjApvNzJuyAqX5b3wn7,Marius Gebbing
15,6000,mizusize@gmail.com,0x3296B13B1eE66D953d4449BD81581a4612ECc27F,AKIHIRO MIZUGUCHI
...,...,...,...,...
349,1000000,shahsaumil@gmail.com,0x0230c0F62a61D77c62C48B162F6dec2Be8DC4D20,SAUMIL G SHAH
350,350000,b_bordenave@live.com,0x06f62B8B2eebdE73dd4F0E1fA9b1321D63fd5c3A,Benjamin Bordenave
352,345000,ltmemories@gmail.com,0xD5aBc758D753B68985B2Bb83F2b2F4030179ea20,John Gravagna
353,50000,seleznyov@gmail.com,0x0CEAd33EB6AdF4860B6b8F66687f51567E111973,ANDRII SELEZNOV


In [8]:
#get invalid addresses (not long enough)
df2 = df2.astype({'wallet_address': 'string'})
df2["address_length"] = df2.wallet_address.progress_apply(len)
df2["address_length"]=df2["address_length"].astype("category")

In [9]:
#delete invalid addresses
df2 = df2[df2.address_length != 34]

In [10]:
df2

,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length
15,6000,mizusize@gmail.com,0x3296B13B1eE66D953d4449BD81581a4612ECc27F,AKIHIRO MIZUGUCHI,42
16,100,carjean02@gmail.com,0x090Dcf87443E1495b0fFAF1f3B7a1d8b2C91Aa3b,Jeancarlos Acosta Mejia,42
17,35000,s.d.tolpa@web.de,0xAa9D5a6cDb9344fD22cdA14e5DE5c1DE9c41Cc91,SASCHA MRONGOWIUS,42
18,100000,cde@posteo.net,0x9329c22f63898E18e8A67C582FE8092203B90c64,CLAUDIO SEBASTIAN WECK,42
19,2500,crisgti580@gmail.com,0xbEFb4feCd83185D358B891f12eda1c20FdBa9c20,IONUT CRISTIAN MIHAIL,42
...,...,...,...,...,...
349,1000000,shahsaumil@gmail.com,0x0230c0F62a61D77c62C48B162F6dec2Be8DC4D20,SAUMIL G SHAH,42
350,350000,b_bordenave@live.com,0x06f62B8B2eebdE73dd4F0E1fA9b1321D63fd5c3A,Benjamin Bordenave,42
352,345000,ltmemories@gmail.com,0xD5aBc758D753B68985B2Bb83F2b2F4030179ea20,John Gravagna,42
353,50000,seleznyov@gmail.com,0x0CEAd33EB6AdF4860B6b8F66687f51567E111973,ANDRII SELEZNOV,42


In [11]:
df2['sow_estimated_net_worth_value']= pd.to_numeric(df2['sow_estimated_net_worth_value'])

In [12]:
#create a unique hash of the eth address as a uniuq identifier
df2['hash'] = df2.progress_apply(lambda row: hash(row.wallet_address), axis = 1)

In [13]:
#control identifier
hash(df2.iloc[0,2])

-3759754548358546723

## feature engineering


### create a dummy for accredited investor

In [14]:
for i in list(range(0, 304)):
    if df2.iloc[i,0]>=1000000: df2.iloc[i,0]= 1
    else: df2.iloc[i,0]= 0

In [15]:
#check how many accredited investors
df2.iloc[:,0].value_counts()

1.0    163
0.0    141
Name: sow_estimated_net_worth_value, dtype: int64

In [16]:
df2.iloc[24,:]

0
sow_estimated_net_worth_value                                             0
address                                             elmerfudd2019@gmail.com
wallet_address                   0x95EBa9c1E94A0b491b5d5AFd6DcDe49B15e89594
full_name                                                    Declan McGrath
address_length                                                           42
hash                                                    2920318063947896994
Name: 53, dtype: object

In [17]:
#create a copy of df2
df3 = df2

In [18]:
#lowered the address string as it is done that way automatically when used in the etherscan 'API'
df3.wallet_address = list(map(str.lower,df2.wallet_address))
bol = df3.wallet_address == "0x58032c2fd021473eeee3c076bb83c3ee68a15064"
bol.value_counts()

False    303
True       1
Name: wallet_address, dtype: int64

In [19]:
#there was an error due to a wrong wallet_address -> it was after "0x58032c2fd021473eeee3c076bb83c3ee68a15064"
df2.loc[df3.wallet_address == "0x58032c2fd021473eeee3c076bb83c3ee68a15064"]
df2.iloc[range(240,250),:] #take a look at these addresses to find the wrong one
#the wrong address (i: 295) had a different format: bc1qnxsyskg99zx3zfl9mr82tumvnzwdlelryk5wvm instead of 0x58......

,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length,hash
284,1.0,impaladave1966@gmail.com,0xb175655cdf48177a85ec6f62227639e0d07b127c,DAVID EUGENE MEEK,42,-3377833480680285200
287,1.0,Emilpadre@gmail.com,0xab305a6f8e29a554f988ed648d7cc3d783086b7e,EMIL RILEY DE VELA PADRE,42,-2484874475375379085
288,1.0,bvankust@gmail.com,0x0cef5c98d46c5d66102d8250a94ed0b4aa05ec17,Bianca Vankust,42,-8323191018622855477
289,1.0,clewfirst@gmail.com,0x9f552ed132bb199493af3e70eefd1750cfe6b26f,Noah Benjamin Isaacson,42,-8111258546661255258
290,1.0,donkino4@naver.com,0x4028ef77b92e58e8eee00e0677f1e2ce707777de,Jaeman kim,42,-3489631135468251931
292,1.0,flydicegames@gmail.com,0xc47b437961e8b280ac46cef885c3c205b26f3ca2,Adam Hackett,42,-1364710527549505249
293,0.0,jp.temperilli@gmail.com,0x5946d65d94cc71c3c8506d27d914cd40763cdf30,John Peter Temperilli,42,-9164582272556392068
294,1.0,johndaviddiaz@hotmail.com,0x58032c2fd021473eeee3c076bb83c3ee68a15064,John David Diaz,42,6095026826925048794
295,1.0,hamojona@gmail.com,bc1qnxsyskg99zx3zfl9mr82tumvnzwdlelryk5wvm,JONATHAN HAMO,42,-4142642728904074142
296,1.0,chad.cromwell@gmail.com,0x21502394e8fec1d65c82d848c1af3c5ca41b2461,Chad Anthony Cromwell,42,4731094895008983929


In [20]:
#delete the wrong address
df3 = df3[df3.wallet_address != "bc1qnxsyskg99zx3zfl9mr82tumvnzwdlelryk5wvm"]
df3.iloc[range(240,250),:] #check if successfull

,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length,hash
284,1.0,impaladave1966@gmail.com,0xb175655cdf48177a85ec6f62227639e0d07b127c,DAVID EUGENE MEEK,42,-3377833480680285200
287,1.0,Emilpadre@gmail.com,0xab305a6f8e29a554f988ed648d7cc3d783086b7e,EMIL RILEY DE VELA PADRE,42,-2484874475375379085
288,1.0,bvankust@gmail.com,0x0cef5c98d46c5d66102d8250a94ed0b4aa05ec17,Bianca Vankust,42,-8323191018622855477
289,1.0,clewfirst@gmail.com,0x9f552ed132bb199493af3e70eefd1750cfe6b26f,Noah Benjamin Isaacson,42,-8111258546661255258
290,1.0,donkino4@naver.com,0x4028ef77b92e58e8eee00e0677f1e2ce707777de,Jaeman kim,42,-3489631135468251931
292,1.0,flydicegames@gmail.com,0xc47b437961e8b280ac46cef885c3c205b26f3ca2,Adam Hackett,42,-1364710527549505249
293,0.0,jp.temperilli@gmail.com,0x5946d65d94cc71c3c8506d27d914cd40763cdf30,John Peter Temperilli,42,-9164582272556392068
294,1.0,johndaviddiaz@hotmail.com,0x58032c2fd021473eeee3c076bb83c3ee68a15064,John David Diaz,42,6095026826925048794
296,1.0,chad.cromwell@gmail.com,0x21502394e8fec1d65c82d848c1af3c5ca41b2461,Chad Anthony Cromwell,42,4731094895008983929
297,0.0,djvcuda@comcast.net,0x4fa537718c024a7d61f3168422682cc97bb52234,David Verna,42,-7092676412563308741


In [21]:
#delecte all "non-ETH" addresses -> they don´t start with 0x
start, stop = 0, 2
df3["address_beginning"]= df3["wallet_address"].str.slice(start, stop)
#use raw strings -> also handles backlashes as normal characters done by r"string" instead of "string"
df3["address_beginning"] = df3["address_beginning"].astype("category")

<ipython-input-21-3bf06a9bd693>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["address_beginning"]= df3["wallet_address"].str.slice(start, stop)
<ipython-input-21-3bf06a9bd693>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["address_beginning"] = df3["address_beginning"].astype("category")


In [22]:
df3.address_beginning.value_counts()

0x    303
Name: address_beginning, dtype: int64

### get normal transaction data

In [23]:
import requests
from decimal import Decimal 
import certifi
from collections import defaultdict
from datetime import datetime

In [24]:
def list_to_string(_list):
        return ",".join(_list) if isinstance(_list, list) else _list

In [25]:
from time import sleep

def api_call(addresses, numbers_transactions, api_key):    
    # url placeholders
    base_url = "https://api.etherscan.io/api"
    url = f"{base_url}?module=account&action=txlist&address={list_to_string(addresses)}&startblock=0&endblock=99999999&page=1&offset={str(numbers_transactions)}&sort=asc&apikey={api_key}"

    try:
        response = requests.get(url, verify=certifi.where())

        #print(response.status_code)
        if response.status_code == 200:
            address_content = response.json()
            
            if address_content.get("message") == "NOTOK":
                return {}

            # print(1) 
            #1 == 1
            #sleep(1) # TODO this is a hack to prevent 'DNS Erros / Connection refused Erros after a ton of request to the api                
            return address_content.get("result")#list of dictionaries
        else:
            return {}
    except Exception as error:
        return {}
        


In [26]:
result = api_call('0xab305a6f8e29a554f988ed648d7cc3d783086b7e', 10, api_key='QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ')
print(result)

[{'blockNumber': '12427316', 'timeStamp': '1620925923', 'hash': '0xdab180d8f9b863d356e4744480c6fbc028e9511ab6ea4b87f065c02ecea0c8da', 'nonce': '1877451', 'blockHash': '0x4254fdf695cecf26a431665ed6ab27b3f131482a085e2d44127714fddac70790', 'transactionIndex': '116', 'from': '0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0', 'to': '0xab305a6f8e29a554f988ed648d7cc3d783086b7e', 'value': '120920000000000000', 'gas': '30000', 'gasPrice': '168000000000', 'isError': '0', 'txreceipt_status': '1', 'input': '0x', 'contractAddress': '', 'cumulativeGasUsed': '7075006', 'gasUsed': '21000', 'confirmations': '746383'}, {'blockNumber': '12432682', 'timeStamp': '1620997473', 'hash': '0x8c8df3ea56f1d46a0ac0b9b7a2d3e33a2fd44de5cf5dc775dcf927bd6c0b6cf4', 'nonce': '0', 'blockHash': '0x29d129445ab20dc53877e2e34b378b172c6a6136456676515a62215d1638aa73', 'transactionIndex': '208', 'from': '0xab305a6f8e29a554f988ed648d7cc3d783086b7e', 'to': '0xe592427a0aece92de3edee1f18e0157c05861564', 'value': '95909377525292500', 'ga

In [27]:
#get the data per address (hier funktioniert es nur mit einer adresse)
def get_transactions_by_address(address, numbers_transactions, api_key='QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ'):
    data = defaultdict(dict)

    try: # this makes sure the programm is not crashing; even if the api call failes
        result = api_call(address, numbers_transactions, api_key)

    except Exception as error:
        print(error)

        data[0]["hash"] = "" #hash of the transaction
        data[0]["tx_from"] = "" #sender address
        data[0]["tx_to"] = "" #receiver
        data[0]["eth_value"] = 0 #value in eth
        data[0]["out"] = False # dummy - true if the address is sending the money
        data[0]["confirmed"] = False # if the transaction is confirmed
        data[0]["timeStamp"] = datetime.strptime("2000-08-01 12:30", '%Y-%m-%d %H:%M')
        data[0]["nonce"] = 0 #number of confirmed transactions originated from this address
        data[0]["gas"] = 0 #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[0]["will_gas_Price"] = 0 #How much the transaction originator is willing t. pay in Wei per gas unit
        data[0]["gasUsed"] = 0 #How much gas was used at the end
        data[0]["transaction_cost"] = 0
        data[0]["data_transaction"] = ""
        data[0]["isError"] = 0
        data[0]["cumulativeGasUsed"] = 0
        return data

    
    for n, transaction in enumerate(result):
        hashs = transaction.get("hash")
        tx_from = transaction.get("from")
        tx_to = transaction.get("to")
        value = transaction.get("value")
        confirmations = transaction.get("confirmations")
        time = datetime.utcfromtimestamp(int(transaction.get("timeStamp"))).strftime('%Y-%m-%d %H:%M')
        nonce = transaction.get("nonce")
        gas = float(transaction.get("gas"))
        gasPrice = float(transaction.get("gasPrice"))
        gasUsed = transaction.get("gasUsed")
        datatx = transaction.get("input")
        error = transaction.get("isError")
        cumulativeGasUsed = transaction.get("cumulativeGasUsed")
        
        if tx_from == address:
            out = True
        else: 
            out = False

        eth_value = Decimal(value) / Decimal("1000000000000000000") #float test sodass Zahl
        
        gasprice_eth = Decimal(gasPrice) / Decimal("1000000000000000000")
        transaction_cost = Decimal(gasprice_eth) * Decimal(gasUsed)
        if int(confirmations) >=16:
            confirmed = True
        else:
            confirmed = False
        
        data[n]["hash"] = hashs #hash of the transaction
        data[n]["tx_from"] = tx_from #sender address
        data[n]["tx_to"] = tx_to #receiver
        data[n]["eth_value"] = eth_value #value in eth
        data[n]["out"] = out # dummy - true if the address is sending the money
        data[n]["confirmed"] = confirmed # if the transaction is confirmed
        data[n]["timeStamp"] = datetime.strptime(time, '%Y-%m-%d %H:%M')
        data[n]["nonce"] = nonce #number of confirmed transactions originated from this address
        data[n]["gas"] = gas #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[n]["will_gas_Price"] = gasprice_eth #How much the transaction originator is willing t. pay in Wei per gas unit
        data[n]["gasUsed"] = gasUsed #How much gas was used at the end
        data[n]["transaction_cost"] = transaction_cost
        data[n]["data_transaction"] = datatx
        data[n]["isError"] = error
        data[n]["cumulativeGasUsed"] = cumulativeGasUsed
    
    return data    
    #reminder: die Berechnungen müssen für´s Umrechnen in Float übertragen werden

    

In [28]:
#create a data frame that consists out of all the transactions of the whole addresses
new_df = df3.wallet_address.progress_apply(get_transactions_by_address, numbers_transactions = 1500)

In [29]:
def normalize_data(df):
    temp_list = []
    for key, value in df.items():
        for k,v in value.items():
            temp_list.append(v)

    return pd.DataFrame(temp_list)

In [30]:
new_df

15     {0: {'hash': '0x9255fc5f3aebc183fad66c2cdc8723...
16                                                    {}
17     {0: {'hash': '0xbf0d443dc9ded9b098c3812b83a3bf...
18     {0: {'hash': '0x1d2e98dd3ede3f1dbe9e6b29c5714e...
19     {0: {'hash': '0xda34951f77ec05f0ce784d428a5011...
                             ...                        
349    {0: {'hash': '0x5ca0a02cae1207eb42d2621faa508e...
350    {0: {'hash': '0xef3fc2a2b6886592bab40885e8b5b9...
352                                                   {}
353    {0: {'hash': '0xb556e12c1502ae9de3e0b5322eb1c3...
354    {0: {'hash': '0x2b1909e41cf3551ca1baffe1ce0b11...
Name: wallet_address, Length: 303, dtype: object

In [32]:
transaction_df = normalize_data(new_df)
transaction_df # That´s the final edge list of the normal TX

,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,will_gas_Price,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed
0,0x9255fc5f3aebc183fad66c2cdc8723725b58bc8fdae4...,0x003e36550908907c2a2da960fd19a419b9a774b7,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0.004193528312830012,False,True,2018-05-15 08:09:00,48351,21000.0,1E-8,21000,0.00021000,0x,0,418019
1,0xdf5ccd2c08ecd7bc8c07fdb936ba183e06fa2a000a40...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.001941142112645209,True,True,2018-05-15 08:21:00,0,158820.0,1E-8,60508,0.00060508,0x454a2ab3000000000000000000000000000000000000...,0,1889733
2,0x743f000df6bbd13a48fcc02d84adf26fd2a8bc84666b...,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0.04000044,False,True,2018-05-17 07:11:00,2362530,21000.0,5E-8,21000,0.00105000,0x,0,3711616
3,0xb1abcc5b22c2f1fee9d1591a7060842e44b14586a385...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.002333140045277691,True,True,2018-05-17 07:58:00,1,135963.0,1E-8,45508,0.00045508,0x454a2ab3000000000000000000000000000000000000...,0,7476882
4,0x685ff133d98dd4da61bc0ac7e85c52d4aa7b3debd1b3...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x06012c8cf97bead5deae237070f9587f8e7a266d,0.008,True,True,2018-05-17 08:15:00,2,119977.0,1E-8,79985,0.00079985,0xf7d8c883000000000000000000000000000000000000...,0,3803770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77184,0x2178db4f0b76fac2d79f526063618682221dd84a11c2...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xe592427a0aece92de3edee1f18e0157c05861564,0,True,True,2021-09-03 22:17:00,1399,182871.0,3.62397871754E-7,126236,0.045747657738737944,0xac9650d8000000000000000000000000000000000000...,0,18505183
77185,0x0240c54432bb187903e4690fc148e9b6c853441cff77...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,True,True,2021-09-04 05:59:00,1400,199731.0,1.12428831545E-7,137061,0.015409608080389245,0x18cbafe5000000000000000000000000000000000000...,0,11166105
77186,0x0f102bc9316d054f863ef09c1562bd466a0f8206214b...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x5603172654f37b509d74c6fe3a3fc087104100d9,0.3,True,True,2021-09-04 20:24:00,1401,1097939.0,9.2918128265E-8,742949,0.069033430476353485,0xa0712d68000000000000000000000000000000000000...,0,24198575
77187,0x74e5120bd10f4dbff32981b03fe7cf317acdbfc1bb85...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x2ea241b51451b713dd05566d10a6ba3379b26788,0,True,True,2021-09-04 20:29:00,1402,1150227.0,9.0098044006E-8,766818,0.069088801908592908,0x379607f5000000000000000000000000000000000000...,0,20037682


## get the balance of the addresses

In [36]:
def balance_api_call(addresses, api_key):
    # url placeholders
    
    base_url = "https://api.etherscan.io/api"
    url = f"{base_url}?module=account&action=balance&address={list_to_string(addresses)}&tag=latest&apikey={api_key}"

    try:
        response = requests.get(url, verify=certifi.where())

        #print(response.status_code)
        if response.status_code == 200:
            address_content = response.json()
            
            if address_content.get("message") == "NOTOK":
                return {}

            #sleep(1) # TODO this is a hack to prevent 'DNS Erros / Connection refused Erros after a ton of request to the api                
            return address_content.get("result")#list of dictionaries
        else:
            return {}
    except Exception as error:
        print(error)
        return {}

In [386]:
#test API call: 
#-> gives Balance in Gwei at the time of retrieval
result = balance_api_call("0x0230c0f62a61d77c62c48b162f6dec2be8dc4d20", 'QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ')
Decimal(result)

Decimal('11483144030075337')

In [286]:
x= Decimal('2241918556922885')/Decimal("1000000000000000000")
y = x*Decimal("3302")
y

Decimal('7.402815074959366270')

In [37]:
#Get balance for addresses (date: Monday, August 30, 8pm CEST)
address_balance = []

for index, row in df3.iterrows():
    result = balance_api_call(row.wallet_address, "QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ")
    address_balance.append({"address_balance":result})

    
address_balance = pd.DataFrame(address_balance)
df3.loc[:,"address_balance"]= address_balance

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [58]:
df3

,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length,hash,address_beginning,address_balance
15,0.0,mizusize@gmail.com,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,AKIHIRO MIZUGUCHI,42,-4088783140720925316,0x,246167951093634996
16,0.0,carjean02@gmail.com,0x090dcf87443e1495b0ffaf1f3b7a1d8b2c91aa3b,Jeancarlos Acosta Mejia,42,7126748193056654943,0x,359507000000000
17,0.0,s.d.tolpa@web.de,0xaa9d5a6cdb9344fd22cda14e5de5c1de9c41cc91,SASCHA MRONGOWIUS,42,4719250612253798252,0x,246845980595768771
18,0.0,cde@posteo.net,0x9329c22f63898e18e8a67c582fe8092203b90c64,CLAUDIO SEBASTIAN WECK,42,2045020602619597288,0x,5214635318886014489
19,0.0,crisgti580@gmail.com,0xbefb4fecd83185d358b891f12eda1c20fdba9c20,IONUT CRISTIAN MIHAIL,42,-2058215953495933286,0x,454540735692808878
...,...,...,...,...,...,...,...,...
349,1.0,shahsaumil@gmail.com,0x0230c0f62a61d77c62c48b162f6dec2be8dc4d20,SAUMIL G SHAH,42,1231488335706197333,0x,NaN
350,0.0,b_bordenave@live.com,0x06f62b8b2eebde73dd4f0e1fa9b1321d63fd5c3a,Benjamin Bordenave,42,1827549469105083427,0x,NaN
352,0.0,ltmemories@gmail.com,0xd5abc758d753b68985b2bb83f2b2f4030179ea20,John Gravagna,42,-3082015307768915211,0x,NaN
353,0.0,seleznyov@gmail.com,0x0cead33eb6adf4860b6b8f66687f51567e111973,ANDRII SELEZNOV,42,7925451324816931321,0x,NaN


## Get the neighboars of the fetched data


In [65]:
def get_first_neighboars(lookup_address, transactions=transaction_df):
    """
        param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    outgoing_transactions =  transactions[(transactions.tx_from == lookup_person) & (transactions.out == True)]
    outgoing_neighboars = outgoing_transactions.drop_duplicates(subset='tx_to')
    # print(len(outgoing_neighboars))

    incomming_transactions =  transactions[(transactions.tx_to == lookup_person) & (transactions.out == False)]
    incomming_neighboars = incomming_transactions.drop_duplicates(subset='tx_from')
    # print(len(incomming_neighboars))

    list_ = outgoing_neighboars.tx_to.to_list()
    list_ += incomming_neighboars.tx_from.to_list()


    first_neighboars = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars = first_neighboars.drop_duplicates(subset='neighboar')

    return first_neighboars



In [66]:
def get_first_out_neighboars(lookup_address, transactions=transaction_df):
    """
        param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    outgoing_transactions =  transactions[(transactions.tx_from == lookup_person) & (transactions.out == True)]
    outgoing_neighboars = outgoing_transactions.drop_duplicates(subset='tx_to')
    # print(len(outgoing_neighboars))

    list_ = outgoing_neighboars.tx_to.to_list()

    first_neighboars_out = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars_out = first_neighboars_out.drop_duplicates(subset='neighboar')

    return first_neighboars_out

In [67]:
def get_first_neighboars_in(lookup_address, transactions=transaction_df):
    """
        param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    incomming_transactions =  transactions[(transactions.tx_to == lookup_person) & (transactions.out == False)]
    incomming_neighboars = incomming_transactions.drop_duplicates(subset='tx_from')
    # print(len(incomming_neighboars))

    list_ = incomming_neighboars.tx_from.to_list()


    first_neighboars_in = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars_in = first_neighboars_in.drop_duplicates(subset='neighboar')

    return first_neighboars_in




## Get first neighboar for every address in d3

In [68]:
neighboars = []

for index, row in df3.iterrows():
    
    result = get_first_neighboars(row.wallet_address)
    neighboars.append(result)
    
neighboars = pd.concat(neighboars)
neighboars

,neighboar,id
0,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
1,0x06012c8cf97bead5deae237070f9587f8e7a266d,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
2,0x4d39217fd94abf03928da97488b16ac91816108f,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
3,0x199f2202d878e9747b4f96cdf2c980d59d7c0969,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
4,0xdd4678e7f3d7ee45c62a0e3424a7a95d82a86489,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
...,...,...
203,0xddfabcdc4d8ffc6d5beaf154f18b778f892a0740,0x2fe4b878e1fd34e7ec449e7521b8333495608805
204,0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511,0x2fe4b878e1fd34e7ec449e7521b8333495608805
207,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f,0x2fe4b878e1fd34e7ec449e7521b8333495608805
208,0x17efa927462b0541c16e118c10e770c266049376,0x2fe4b878e1fd34e7ec449e7521b8333495608805


In [69]:
neighboars_out = []

for index, row in df3.iterrows():
    
    result = get_first_out_neighboars(row.wallet_address)
    neighboars_out.append(result)
    
neighboars_out = pd.concat(neighboars_out)
neighboars_out

,neighboar,id
0,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
1,0x06012c8cf97bead5deae237070f9587f8e7a266d,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
2,0x4d39217fd94abf03928da97488b16ac91816108f,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
3,0x199f2202d878e9747b4f96cdf2c980d59d7c0969,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
4,0xdd4678e7f3d7ee45c62a0e3424a7a95d82a86489,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
...,...,...
167,0x60e4d786628fea6478f785a6d7e704777c86a7c6,0x2fe4b878e1fd34e7ec449e7521b8333495608805
168,0x22c36bfdcef207f9c0cc941936eff94d4246d14a,0x2fe4b878e1fd34e7ec449e7521b8333495608805
169,0x9757f2d2b135150bbeb65308d4a91804107cd8d6,0x2fe4b878e1fd34e7ec449e7521b8333495608805
170,0xb087ec2cf65fa9372cf61a20602bd37399837985,0x2fe4b878e1fd34e7ec449e7521b8333495608805


In [70]:
neighboars_in = []

for index, row in df3.iterrows():
    
    result = get_first_neighboars_in(row.wallet_address)
    neighboars_in.append(result)
    
neighboars_in = pd.concat(neighboars_in)
neighboars_in

,neighboar,id
0,0x003e36550908907c2a2da960fd19a419b9a774b7,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
1,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
2,0x0681d8db095565fe8a346fa0277bffde9c0edbbf,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
3,0xda92c68565b6f8b4ee5ce619fabd880f6e485604,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
4,0xce85247b032f7528ba97396f7b17c76d5d034d2f,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
...,...,...
33,0xfbe30e8fcb7e2a508dcd5e14a68b0f49ec902857,0x2fe4b878e1fd34e7ec449e7521b8333495608805
34,0x3689dfeca9c89e39b89b740a0336dc345e5d1ec2,0x2fe4b878e1fd34e7ec449e7521b8333495608805
35,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f,0x2fe4b878e1fd34e7ec449e7521b8333495608805
36,0x17efa927462b0541c16e118c10e770c266049376,0x2fe4b878e1fd34e7ec449e7521b8333495608805


In [71]:
neighboars

,neighboar,id
0,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
1,0x06012c8cf97bead5deae237070f9587f8e7a266d,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
2,0x4d39217fd94abf03928da97488b16ac91816108f,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
3,0x199f2202d878e9747b4f96cdf2c980d59d7c0969,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
4,0xdd4678e7f3d7ee45c62a0e3424a7a95d82a86489,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
...,...,...
203,0xddfabcdc4d8ffc6d5beaf154f18b778f892a0740,0x2fe4b878e1fd34e7ec449e7521b8333495608805
204,0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511,0x2fe4b878e1fd34e7ec449e7521b8333495608805
207,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f,0x2fe4b878e1fd34e7ec449e7521b8333495608805
208,0x17efa927462b0541c16e118c10e770c266049376,0x2fe4b878e1fd34e7ec449e7521b8333495608805


## Get transactions for first neighboars

In [ ]:
neighboar_transactions = []

placeholder = []
for i in range(0, len(neighboars)):
    id_ = neighboars.iloc[i].id
    neighboar_ = neighboars.iloc[i].neighboar

    transactions = get_transactions_by_address(neighboar_, numbers_transactions=1600)
    transactions = pd.DataFrame(transactions).transpose()
    transactions["id"] = id_
    transactions["neighboar"] = neighboar_
    placeholder.append(transactions)

first_neighboars_transactions = pd.concat(placeholder)
first_neighboars_transactions


KeyboardInterrupt: 

In [72]:
first_neighboars_transactions.to_csv("first_neighb_transactions.csv")

NameError: name 'first_neighboars_transactions' is not defined

In [73]:
EL_neighb_normal = pd.read_csv("first_neighb_transactions.csv", header = 0)
EL_neighb_normal #edge list of neighbor tx (normal)

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,will_gas_Price,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed,id,neighboar
0,0,0x44fd65a70b4ac20e6b59c10692b5d411c636a318afe3...,0xba52c75764d6f594735dc735be7f1830cdf58ddf,NaN,0.000000,False,True,2017-11-23 05:41:00,3516,4500000.0,4.100000e-09,1078399,0.004421,0x60606040526000805460a060020a60ff021916905560...,0,1793664,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c
1,1,0x5c8a04e3174fed51f8cbe8f8ec77b6d5714343c3ebc6...,0x672ec49f7f7eac25c3a2e651f67f579bb5da8936,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.009966,False,True,2017-11-23 18:46:00,5,158724.0,2.000000e-09,60444,0.000121,0x454a2ab3000000000000000000000000000000000000...,0,4184712,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c
2,2,0x621c58ab6adc225f021cc66cc94003c4cf921f277375...,0x76df4c63c28d462d5064add17f51e2ad55c9d5a2,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.009930,False,True,2017-11-23 18:46:00,2,158724.0,2.000000e-08,60444,0.001209,0x454a2ab3000000000000000000000000000000000000...,0,370276,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c
3,3,0x55687b35a9c82c420da07b504032db56dba544828fd2...,0x730fafe1108f507b454b6eb59f0eec1f1ba31935,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.009882,False,True,2017-11-23 18:48:00,48,200000.0,2.000000e-09,22476,0.000045,0x454a2ab3000000000000000000000000000000000000...,0,3251348,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c
4,4,0xc9812b823ddd7b32184525598aafeae78e3b4544a471...,0x730fafe1108f507b454b6eb59f0eec1f1ba31935,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.009851,False,True,2017-11-23 18:48:00,49,200000.0,2.000000e-09,60444,0.000121,0x454a2ab3000000000000000000000000000000000000...,0,3311792,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21170194,1574,0xdf984072ecdbb41d37ab49d75a161fd81eccae353364...,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0.060000,True,True,2021-08-15 18:56:00,1525,228693.0,4.950000e-08,69291,0.003430,0xab834bab0000000000000000000000007be8076f4ea4...,1,1500670,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f
21170195,1575,0x4a8183c754beb44e8c002985036dfab69cd128739a27...,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0.500000,True,True,2021-08-15 20:00:00,1526,228741.0,6.270000e-08,218735,0.013715,0xab834bab0000000000000000000000007be8076f4ea4...,0,16612338,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f
21170196,1576,0xdb0fe11e7a7e9a14a03750a3ce315589d0b2280919f7...,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f,0xfbeef911dc5821886e1dda71586d90ed28174b7d,0.200000,True,True,2021-08-16 01:46:00,1527,385521.0,5.300000e-08,375521,0.019903,0xefef39a1000000000000000000000000000000000000...,0,12936526,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f
21170197,1577,0x7165f43441bb73daa385d131ac7c7047e86392d6f548...,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.000000,True,True,2021-08-16 04:58:00,1528,36027.0,5.100000e-08,30416,0.001551,0x2e1a7d4d000000000000000000000000000000000000...,0,19002484,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x04c1c15a1554891fb2dd7c4939e7d95d06ee253f


In [183]:
import random
random.seed(10)

EL_small = EL_neighb_normal.sample(frac = 0.1)
len(EL_small)

2117020

## Calculate TX of the normal edge lists in USD

### calc USD for TX original adresses

In [ ]:
transaction_df.hash[1]

'0xdf5ccd2c08ecd7bc8c07fdb936ba183e06fa2a000a408c7f243db6ff5f04c63d'

In [ ]:
transaction_df.iloc[1]

hash                 0xdf5ccd2c08ecd7bc8c07fdb936ba183e06fa2a000a40...
tx_from                     0x3296b13b1ee66d953d4449bd81581a4612ecc27f
tx_to                       0xb1690c08e213a35ed9bab7b318de14420fb57d8c
eth_value                                         0.001941142112645209
out                                                               True
confirmed                                                         True
timeStamp                                          2018-05-15 08:21:00
nonce                                                                0
gas                                                             158820
will_gas_Price                                                    1E-8
gasUsed                                                          60508
transaction_cost                                            0.00060508
data_transaction     0x454a2ab3000000000000000000000000000000000000...
isError                                                              0
cumula

In [ ]:
print(float(transaction_df.eth_value[1])) #0.001941142112645209 from etherscan
float(transaction_df.will_gas_Price[1]) #abgleichen mit ETH Scan
(10000000000/1000000000000000000)*60508 #transaction costs -> ETH scan = 0.00060508

0.001941142112645209


0.00060508

In [7]:
#read in the USD/ETH data
ethUSD= pd.read_csv('ETH_1min.txt', header = None)
ethUSD.columns = ["timeStamp", "Open", "High", "Low", "Close", "Volume"]
ethUSD_s = ethUSD[["timeStamp",  "Close"]].copy()

In [34]:
#redefine the timeStamp column with a string
transaction_df.timeStamp = transaction_df.timeStamp.astype(str)

In [35]:
#merge the exchange and the transaction data with respect to the time Stamp #reminder -> some data older than 2016/03 is lost
df_test = pd.merge(transaction_df, ethUSD_s, on='timeStamp')

In [36]:
#calculate approx. value of transaction parameters in USD
df_test["eth_valueUSD"] = df_test.eth_value.astype("float") * df_test.Close
df_test["transaction_costUSD"] = df_test.transaction_cost.astype("float") * df_test.Close
df_test["will_gas_PriceUSD"] = df_test.will_gas_Price.astype("float") * df_test.Close

In [87]:
#test whether the approximations are decent -> Stichprobe ausstehend mit 30 random transactions
df_test.iloc[1] #get the hash and incert it in etherscan
 #check out the transaction data in the data frame & compare it with etherscan -> not perfect but alright

hash                   0xdf5ccd2c08ecd7bc8c07fdb936ba183e06fa2a000a40...
tx_from                       0x3296b13b1ee66d953d4449bd81581a4612ecc27f
tx_to                         0xb1690c08e213a35ed9bab7b318de14420fb57d8c
eth_value                                           0.001941142112645209
out                                                                 True
confirmed                                                           True
timeStamp                                            2018-05-15 08:21:00
nonce                                                                  0
gas                                                               158820
will_gas_Price                                                      1E-8
gasUsed                                                            60508
transaction_cost                                              0.00060508
data_transaction       0x454a2ab3000000000000000000000000000000000000...
isError                                            

In [98]:
df_test.to_csv("df_origin_norm_tx.csv")

In [3]:
df_test= pd.read_csv("df_origin_norm_tx.csv")

In [14]:
df_test

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,will_gas_Price,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed,Close,eth_valueUSD,transaction_costUSD,will_gas_PriceUSD
0,0,0x9255fc5f3aebc183fad66c2cdc8723725b58bc8fdae4...,0x003e36550908907c2a2da960fd19a419b9a774b7,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0.004194,False,True,2018-05-15 08:09:00,48351,21000.0,1.000000e-08,21000,0.000210,0x,0,418019,733.93,3.077756,0.154125,0.000007
1,1,0xdf5ccd2c08ecd7bc8c07fdb936ba183e06fa2a000a40...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.001941,True,True,2018-05-15 08:21:00,0,158820.0,1.000000e-08,60508,0.000605,0x454a2ab3000000000000000000000000000000000000...,0,1889733,734.90,1.426545,0.444673,0.000007
2,2,0x743f000df6bbd13a48fcc02d84adf26fd2a8bc84666b...,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0.040000,False,True,2018-05-17 07:11:00,2362530,21000.0,5.000000e-08,21000,0.001050,0x,0,3711616,701.35,28.054309,0.736418,0.000035
3,3,0xb1abcc5b22c2f1fee9d1591a7060842e44b14586a385...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.002333,True,True,2018-05-17 07:58:00,1,135963.0,1.000000e-08,45508,0.000455,0x454a2ab3000000000000000000000000000000000000...,0,7476882,699.93,1.633035,0.318524,0.000007
4,4,0x685ff133d98dd4da61bc0ac7e85c52d4aa7b3debd1b3...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x06012c8cf97bead5deae237070f9587f8e7a266d,0.008000,True,True,2018-05-17 08:15:00,2,119977.0,1.000000e-08,79985,0.000800,0xf7d8c883000000000000000000000000000000000000...,0,3803770,701.15,5.609200,0.560815,0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64636,64636,0x8af603cf08160290dd98a9ccc98d31aa916275bc8892...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0.100000,True,True,2021-07-14 09:53:00,992,218853.0,4.000000e-08,208945,0.008358,0xab834bab0000000000000000000000007be8076f4ea4...,0,11442929,1932.79,193.279000,16.153872,0.000077
64637,64637,0xff47934465b8d614da2fc950e317bcf66a8ca9e0fdba...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xebc86fb12ab0ffac6cbcafce2f049bfe7efada0d,0.000000,True,True,2021-07-14 21:23:00,993,200000.0,2.800000e-08,121288,0.003396,0x1bf6ddae000000000000000000000000000000000000...,0,3738598,1991.11,0.000000,6.761937,0.000056
64638,64638,0x42e548c91e06188c3e093db7650b2f665dd945aab37a...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0.000000,True,True,2021-07-15 18:30:00,994,46280.0,4.150000e-08,46280,0.001921,0xa22cb465000000000000000000000000239155fb141a...,0,14614544,1911.14,0.000000,3.670574,0.000079
64639,64639,0xb09ac8e9c51e39e89102268e06101f4da2701013a8a8...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0.000000,True,True,2021-07-15 18:35:00,995,266421.0,4.300000e-08,243842,0.010485,0xab834bab0000000000000000000000007be8076f4ea4...,0,2465627,1910.33,0.000000,20.030204,0.000082


In [21]:
df_test = df_test.drop("Unnamed: 0", axis=1)
df_print = df_test.describe().transpose()

KeyError: "['Unnamed: 0'] not found in axis"

'\\begin{tabular}{rrrrrrrr}\n\\toprule\n   count &          mean &           std &        min &           25\\% &           50\\% &           75\\% &           max \\\\\n\\midrule\n 64641.0 &  1.659370e+00 &  1.864407e+01 &      0.000 &  0.000000e+00 &  0.000000e+00 &  2.000000e-01 &  1.999000e+03 \\\\\n 64641.0 &  8.580783e+04 &  6.177215e+05 &      0.000 &  7.300000e+01 &  2.600000e+02 &  6.860000e+02 &  3.577159e+07 \\\\\n 64641.0 &  1.791412e+05 &  6.151536e+05 &  21000.000 &  5.228600e+04 &  1.508500e+05 &  2.071280e+05 &  1.427783e+07 \\\\\n 64641.0 &  1.020554e-07 &  9.621201e-08 &      0.000 &  3.630000e-08 &  8.060000e-08 &  1.400000e-07 &  5.000000e-06 \\\\\n 64641.0 &  9.522702e+04 &  1.071355e+05 &  14457.000 &  3.407200e+04 &  5.652800e+04 &  1.328350e+05 &  8.612017e+06 \\\\\n 64641.0 &  9.090521e-03 &  1.337455e-02 &      0.000 &  1.999748e-03 &  4.861340e-03 &  1.116378e-02 &  5.575735e-01 \\\\\n 64641.0 &  9.029873e-02 &  2.866115e-01 &      0.000 &  0.000000e+00 &  0.

### calc USD for 1st neighb normal TX 

In [88]:
EL_neighb_normal.timeStamp = EL_neighb_normal.timeStamp.astype(str)

In [90]:
df_Neighb_normal_USD = pd.merge(EL_neighb_normal, ethUSD_s, on='timeStamp')
df_Neighb_normal_USD["eth_valueUSD"] = df_Neighb_normal_USD.eth_value.astype("float") * df_Neighb_normal_USD.Close
df_Neighb_normal_USD["transaction_costUSD"] = df_Neighb_normal_USD.transaction_cost.astype("float") * df_Neighb_normal_USD.Close
df_Neighb_normal_USD["will_gas_PriceUSD"] = df_Neighb_normal_USD.will_gas_Price.astype("float") * df_Neighb_normal_USD.Close

In [91]:
df_Neighb_normal_USD.head() #final EL for normal neighb transactions

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,...,transaction_cost,data_transaction,isError,cumulativeGasUsed,id,neighboar,Close,eth_valueUSD,transaction_costUSD,will_gas_PriceUSD
0,0,0x44fd65a70b4ac20e6b59c10692b5d411c636a318afe3...,0xba52c75764d6f594735dc735be7f1830cdf58ddf,NaN,0.0,False,True,2017-11-23 05:41:00,3516,4500000.0,...,0.004421,0x60606040526000805460a060020a60ff021916905560...,0,1793664,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,399.01,0.0,1.764197,0.000002
1,0,0x691f348ef11e9ef95d540a2da2c5f38e36072619aa44...,0xba52c75764d6f594735dc735be7f1830cdf58ddf,NaN,0.0,False,True,2017-11-23 05:41:00,3515,4500000.0,...,0.015455,0x606060409081526002805460a060020a60ff02191690...,0,5959337,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x06012c8cf97bead5deae237070f9587f8e7a266d,399.01,0.0,6.166842,0.000002
2,0,0x44fd65a70b4ac20e6b59c10692b5d411c636a318afe3...,0xba52c75764d6f594735dc735be7f1830cdf58ddf,NaN,0.0,False,True,2017-11-23 05:41:00,3516,4500000.0,...,0.004421,0x60606040526000805460a060020a60ff021916905560...,0,1793664,0x6bf97f2534be2242ddb3a29bfb24d498212dcded,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,399.01,0.0,1.764197,0.000002
3,0,0x691f348ef11e9ef95d540a2da2c5f38e36072619aa44...,0xba52c75764d6f594735dc735be7f1830cdf58ddf,NaN,0.0,False,True,2017-11-23 05:41:00,3515,4500000.0,...,0.015455,0x606060409081526002805460a060020a60ff02191690...,0,5959337,0x6bf97f2534be2242ddb3a29bfb24d498212dcded,0x06012c8cf97bead5deae237070f9587f8e7a266d,399.01,0.0,6.166842,0.000002
4,0,0x44fd65a70b4ac20e6b59c10692b5d411c636a318afe3...,0xba52c75764d6f594735dc735be7f1830cdf58ddf,NaN,0.0,False,True,2017-11-23 05:41:00,3516,4500000.0,...,0.004421,0x60606040526000805460a060020a60ff021916905560...,0,1793664,0x7511b5381f6d06725709f416ea97f9cf7c8fceb2,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,399.01,0.0,1.764197,0.000002


In [186]:
import random
random.seed(10)

small_Neighb_normal_USD = df_Neighb_normal_USD.sample(frac = 0.1)
len(small_Neighb_normal_USD)

1940943

# Get ERC20 TX

In [48]:
from time import sleep

def erc20_api_call(addresses, numbers_transactions, api_key):
    # url placeholders
    
    base_url = "https://api.etherscan.io/api"
    url = f"{base_url}?module=account&action=tokentx&address={list_to_string(addresses)}&startblock=0&endblock=99999999&page=1&offset={str(numbers_transactions)}&sort=asc&apikey={api_key}"

    try:
        response = requests.get(url, verify=certifi.where())

        #print(response.status_code)
        if response.status_code == 200:
            address_content = response.json()
            
            if address_content.get("message") == "NOTOK":
                return {}

            #sleep(1) # TODO this is a hack to prevent 'DNS Erros / Connection refused Erros after a ton of request to the api                
            return address_content.get("result")#list of dictionaries
        else:
            return {}
    except Exception as error:
        print(error)
        return {}
    

In [49]:
#test API Call
erc20_api_call("0xab305a6f8e29a554f988ed648d7cc3d783086b7e", 10000, 'QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ')

[{'blockNumber': '12432682',
  'timeStamp': '1620997473',
  'hash': '0x8c8df3ea56f1d46a0ac0b9b7a2d3e33a2fd44de5cf5dc775dcf927bd6c0b6cf4',
  'nonce': '0',
  'blockHash': '0x29d129445ab20dc53877e2e34b378b172c6a6136456676515a62215d1638aa73',
  'from': '0xb773a5a7ee006d2675537588e3233ad37be53bb9',
  'contractAddress': '0xf1a91c7d44768070f711c68f33a7ca25c8d30268',
  'to': '0xab305a6f8e29a554f988ed648d7cc3d783086b7e',
  'value': '115000000000000000000',
  'tokenName': 'CHARLI3',
  'tokenSymbol': 'C3',
  'tokenDecimal': '18',
  'transactionIndex': '208',
  'gas': '204902',
  'gasPrice': '112000000000',
  'gasUsed': '146760',
  'cumulativeGasUsed': '14721720',
  'input': 'deprecated',
  'confirmations': '718797'},
 {'blockNumber': '12519512',
  'timeStamp': '1622160792',
  'hash': '0x768fdeb784c98d35816ddee2d5c1e2e098d500ceb0ca8553eba5858fe31207d0',
  'nonce': '1',
  'blockHash': '0x49f1a66fb4f73acd4adac71769a53c0915d1bdaae9f0e0b117d7e13e6c0e87d3',
  'from': '0x74de5d4fcbf63e00296fd95d33236b97

In [50]:
#create tx function for getting erc20 tokens 
def get_erc20_token_by_address(address, numbers_transactions, api_key='QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ'):
    data = defaultdict(dict)

    try: # this makes sure the programm is not crashing; even if the api call failes
        result = erc20_api_call(address, numbers_transactions, api_key)

    except Exception as error:
        print(error)

        data[0]["hash"] = "" #hash of the transaction
        data[0]["tx_from"] = "" #sender address
        data[0]["tx_to"] = "" #receiver
        data[0]["eth_value"] = 0 #value in eth
        data[0]["out"] = False # dummy - true if the address is sending the money
        data[0]["confirmed"] = False # if the transaction is confirmed
        data[0]["timeStamp"] = datetime.strptime("2000-08-01 12:30", '%Y-%m-%d %H:%M')
        data[0]["nonce"] = 0 #number of confirmed transactions originated from this address
        data[0]["gas"] = 0 #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[0]["will_gas_Price"] = 0 #How much the transaction originator is willing t. pay in Wei per gas unit
        data[0]["gasUsed"] = 0 #How much gas was used at the end
        data[0]["transaction_cost"] = 0
        data[0]["data_transaction"] = ""
        data[0]["isError"] = 0
        data[0]["cumulativeGasUsed"] = 0
        data[0]["tokenName"] = ""
        data[0]["tokenSymbol"] = ""
        data[0]["tokenDecimal"] = 0
        return data

    
    for n, transaction in enumerate(result):
        hashs = transaction.get("hash")
        tx_from = transaction.get("from")
        tx_to = transaction.get("to")
        value = transaction.get("value")
        confirmations = transaction.get("confirmations")
        time = datetime.utcfromtimestamp(int(transaction.get("timeStamp"))).strftime('%Y-%m-%d %H:%M')
        nonce = transaction.get("nonce")
        gas = float(transaction.get("gas"))
        gasPrice = float(transaction.get("gasPrice"))
        gasUsed = transaction.get("gasUsed")
        datatx = transaction.get("input")
        error = transaction.get("isError")
        cumulativeGasUsed = transaction.get("cumulativeGasUsed")
        tokenName = transaction.get("tokenName")
        tokenSymbol = transaction.get("tokenSymbol")
        tokenDecimal = transaction.get("tokenDecimal")
        
        if tx_from == address:
            out = True
        else: 
            out = False

        eth_value = Decimal(value) / Decimal("1000000000000000000") #float test sodass Zahl
        
        gasprice_eth = Decimal(gasPrice) / Decimal("1000000000000000000")
        transaction_cost = Decimal(gasprice_eth) * Decimal(gasUsed)
        if int(confirmations) >=16:
            confirmed = True
        else:
            confirmed = False
        
        data[n]["hash"] = hashs #hash of the transaction
        data[n]["tx_from"] = tx_from #sender address
        data[n]["tx_to"] = tx_to #receiver
        data[n]["eth_value"] = eth_value #value in eth
        data[n]["out"] = out # dummy - true if the address is sending the money
        data[n]["confirmed"] = confirmed # if the transaction is confirmed
        data[n]["timeStamp"] = datetime.strptime(time, '%Y-%m-%d %H:%M')
        data[n]["nonce"] = nonce #number of confirmed transactions originated from this address
        data[n]["gas"] = gas #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[n]["will_gas_Price"] = gasprice_eth #How much the transaction originator is willing t. pay in Wei per gas unit
        data[n]["gasUsed"] = gasUsed #How much gas was used at the end
        data[n]["transaction_cost"] = transaction_cost
        data[n]["data_transaction"] = datatx
        data[n]["isError"] = error
        data[n]["cumulativeGasUsed"] = cumulativeGasUsed
        data[n]["tokenName"] = tokenName
        data[n]["tokenSymbol"] = tokenSymbol
        data[n]["tokenDecimal"] = tokenDecimal
    
    return data    
    #reminder: die Berechnungen müssen für´s Umrechnen in Float übertragen werden

In [46]:
#test
get_erc20_token_by_address("0xab305a6f8e29a554f988ed648d7cc3d783086b7e", 10000)

name 'erc20_api_call' is not defined


defaultdict(dict,
            {0: {'hash': '',
              'tx_from': '',
              'tx_to': '',
              'eth_value': 0,
              'out': False,
              'confirmed': False,
              'timeStamp': datetime.datetime(2000, 8, 1, 12, 30),
              'nonce': 0,
              'gas': 0,
              'will_gas_Price': 0,
              'gasUsed': 0,
              'transaction_cost': 0,
              'data_transaction': '',
              'isError': 0,
              'cumulativeGasUsed': 0,
              'tokenName': '',
              'tokenSymbol': '',
              'tokenDecimal': 0}})

In [51]:
erc20_df = df3.wallet_address.progress_apply(get_erc20_token_by_address, numbers_transactions = 1500)

In [52]:
erc20_df = normalize_data(erc20_df) #edge list of erc20 transaction of the given addreses

In [99]:
erc20_df.to_csv("erc20_normal_tx.csv")

In [1]:
import pandas as pd
erc20_df = pd.read_csv("erc20_normal_tx.csv")
len(erc20_df)

64722

## Get first neighboar of ERC20 Transaktions

In [53]:
def get_first_ercneighboars(lookup_address, transactions=erc20_df):
    """
        #param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    outgoing_transactions =  transactions[(transactions.tx_from == lookup_person) & (transactions.out == True)]
    outgoing_neighboars = outgoing_transactions.drop_duplicates(subset='tx_to')
    # print(len(outgoing_neighboars))

    incomming_transactions =  transactions[(transactions.tx_to == lookup_person) & (transactions.out == False)]
    incomming_neighboars = incomming_transactions.drop_duplicates(subset='tx_from')
    # print(len(incomming_neighboars))

    list_ = outgoing_neighboars.tx_to.to_list()
    list_ += incomming_neighboars.tx_from.to_list()


    first_neighboars = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars = first_neighboars.drop_duplicates(subset='neighboar')

    return first_neighboars




In [54]:
erc20_neighboars = []

for index, row in df3.iterrows():
    
    result = get_first_ercneighboars(row.wallet_address, transactions=erc20_df)
    erc20_neighboars.append(result)
    
erc20_neighboars = pd.concat(erc20_neighboars)
erc20_neighboars

,neighboar,id
0,0x4d39217fd94abf03928da97488b16ac91816108f,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
1,0x3072d320d538919775e3c8feea6b2fb5721d2693,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
2,0x1eb00f5564d17b2ff40201f7505aa283cb5b7eb4,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
3,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
4,0xd819e948b14ca6aad2b7ffd333ccdf732b129eed,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
...,...,...
267,0x9e35da209b7b1410e00a5e6349462d343d934e90,0x2fe4b878e1fd34e7ec449e7521b8333495608805
268,0x468046386e5f0c49ce8e20d700bf246a8eacbfd1,0x2fe4b878e1fd34e7ec449e7521b8333495608805
269,0xf12ce5503e4bedb7178951321925e4306c4c3b12,0x2fe4b878e1fd34e7ec449e7521b8333495608805
270,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,0x2fe4b878e1fd34e7ec449e7521b8333495608805


In [55]:
erc20_neighboars

,neighboar,id
0,0x4d39217fd94abf03928da97488b16ac91816108f,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
1,0x3072d320d538919775e3c8feea6b2fb5721d2693,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
2,0x1eb00f5564d17b2ff40201f7505aa283cb5b7eb4,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
3,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
4,0xd819e948b14ca6aad2b7ffd333ccdf732b129eed,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
...,...,...
267,0x9e35da209b7b1410e00a5e6349462d343d934e90,0x2fe4b878e1fd34e7ec449e7521b8333495608805
268,0x468046386e5f0c49ce8e20d700bf246a8eacbfd1,0x2fe4b878e1fd34e7ec449e7521b8333495608805
269,0xf12ce5503e4bedb7178951321925e4306c4c3b12,0x2fe4b878e1fd34e7ec449e7521b8333495608805
270,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,0x2fe4b878e1fd34e7ec449e7521b8333495608805


## get tx of first neighb of ERC20 tx

In [63]:
erc_neighboar_transactions = []

erc_placeholder = []
for i in range(0, len(erc20_neighboars)):
    id_ = erc20_neighboars.iloc[i].id
    erc_neighboar_ = erc20_neighboars.iloc[i].neighboar

    transactions = get_erc20_token_by_address(erc_neighboar_, numbers_transactions=1600)
    transactions = pd.DataFrame(transactions).transpose()
    transactions["id"] = id_
    transactions["neighboar"] = erc_neighboar_
    erc_placeholder.append(transactions)

erc_first_neighboars_transactions = pd.concat(erc_placeholder)
erc_first_neighboars_transactions.iloc[] #edgelist of neighb ERC20 tx



('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', TimeoutError(60, 'Operation timed out'))
('Connection aborted.', C

,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,will_gas_Price,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed,tokenName,tokenSymbol,tokenDecimal,id,neighboar
0,0xeef7c5204ac125cb0975e2a111881d34607f006c9b0d...,0x7769f394f87204df364b219821a068989dc19302,0x4d39217fd94abf03928da97488b16ac91816108f,2E-7,False,True,2018-01-19 11:05:00,1555,3.12386e+06,2.5E-8,3023508,0.075587700,deprecated,None,5767884,ArtisTurba,ARTIS,8,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
1,0x66e267f021407d5d348825f00fab3a508d40ba5bbbcb...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1000,False,True,2018-06-20 06:39:00,12,52909,4.1E-8,52909,0.002169269,deprecated,None,1924647,Pundi X Token,NPXS,18,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
2,0xa365ef5b0ec87c8b577218d558626ef566adecfc8d55...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,168096.672827255159765906,False,True,2018-06-20 06:46:00,13,38101,4.1E-8,23101,0.000947141,deprecated,None,1802212,Pundi X Token,NPXS,18,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
3,0x94f95a7eb4ace9415e4e6758c84b8076e8be8547711f...,0x4d39217fd94abf03928da97488b16ac91816108f,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,169096.672827255159765906,True,True,2018-06-20 09:21:00,2,76202,2E-8,23101,0.00046202,deprecated,None,1495070,Pundi X Token,NPXS,18,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
4,0xef21fa7de4a374194917ff0746b711663444fac88d5d...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,6.270493472E-9,False,True,2019-07-15 11:23:00,64,77231,1.265E-8,39733,0.00050262245,deprecated,None,2683575,USD Coin,USDC,6,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,0xce5b19aa87f1eb2696871ad55a941afb1938ba102a02...,0xf12ce5503e4bedb7178951321925e4306c4c3b12,0x4f7714430e8853d238113ee990dbdc299e67295f,800000,True,True,2021-08-14 17:32:00,106,312621,8.3496065949E-8,284201,0.023729665438771749,deprecated,None,562034,SupDao.com,SupDao.com,18,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xf12ce5503e4bedb7178951321925e4306c4c3b12
0,0xa24f9c350906f1886d8bd964d1b35e05b9273aaf2a74...,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0.06,True,True,2021-08-19 02:34:00,1313,310087,4.1E-8,229216,0.009397856,deprecated,None,19131429,Wrapped Ether,WETH,18,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba
1,0xc5354b60bbaa787d918b7650d40fc5461b0bac63fc12...,0xd9cfa54cc68ee6b67fb35b64e3a452c9c423e38f,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,3038661.469476784677103386,False,True,2021-08-22 18:28:00,18,171471,4.8E-8,139556,0.006698688,deprecated,None,245919,BAYC History -The Great Floor Sweep,SWEEP,18,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba
2,0x26384f4968c166fc110f0ad4ce241c115913244aa003...,0xd9cfa54cc68ee6b67fb35b64e3a452c9c423e38f,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,1955085.083757403916373542,False,True,2021-08-22 18:32:00,19,152785,3.7E-8,122578,0.004535386,deprecated,None,1332380,BAYC History -The Great Floor Sweep,SWEEP,18,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba


In [64]:
erc_first_neighboars_transactions.to_csv("erc_first_neighboars_transactions.csv")


,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,...,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed,tokenName,tokenSymbol,tokenDecimal,id,neighboar
0,0,0xeef7c5204ac125cb0975e2a111881d34607f006c9b0d...,0x7769f394f87204df364b219821a068989dc19302,0x4d39217fd94abf03928da97488b16ac91816108f,2.000000e-07,False,True,2018-01-19 11:05:00,1555,3123860.0,...,3023508,0.075588,deprecated,NaN,5767884,ArtisTurba,ARTIS,8.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
1,1,0x66e267f021407d5d348825f00fab3a508d40ba5bbbcb...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1.000000e+03,False,True,2018-06-20 06:39:00,12,52909.0,...,52909,0.002169,deprecated,NaN,1924647,Pundi X Token,NPXS,18.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
2,2,0xa365ef5b0ec87c8b577218d558626ef566adecfc8d55...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1.680967e+05,False,True,2018-06-20 06:46:00,13,38101.0,...,23101,0.000947,deprecated,NaN,1802212,Pundi X Token,NPXS,18.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
3,3,0x94f95a7eb4ace9415e4e6758c84b8076e8be8547711f...,0x4d39217fd94abf03928da97488b16ac91816108f,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,1.690967e+05,True,True,2018-06-20 09:21:00,2,76202.0,...,23101,0.000462,deprecated,NaN,1495070,Pundi X Token,NPXS,18.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
4,4,0xef21fa7de4a374194917ff0746b711663444fac88d5d...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,6.270493e-09,False,True,2019-07-15 11:23:00,64,77231.0,...,39733,0.000503,deprecated,NaN,2683575,USD Coin,USDC,6.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19973355,1599,0xce5b19aa87f1eb2696871ad55a941afb1938ba102a02...,0xf12ce5503e4bedb7178951321925e4306c4c3b12,0x4f7714430e8853d238113ee990dbdc299e67295f,8.000000e+05,True,True,2021-08-14 17:32:00,106,312621.0,...,284201,0.023730,deprecated,NaN,562034,SupDao.com,SupDao.com,18.0,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xf12ce5503e4bedb7178951321925e4306c4c3b12
19973356,0,0xa24f9c350906f1886d8bd964d1b35e05b9273aaf2a74...,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,0x2fe4b878e1fd34e7ec449e7521b8333495608805,6.000000e-02,True,True,2021-08-19 02:34:00,1313,310087.0,...,229216,0.009398,deprecated,NaN,19131429,Wrapped Ether,WETH,18.0,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba
19973357,1,0xc5354b60bbaa787d918b7650d40fc5461b0bac63fc12...,0xd9cfa54cc68ee6b67fb35b64e3a452c9c423e38f,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,3.038661e+06,False,True,2021-08-22 18:28:00,18,171471.0,...,139556,0.006699,deprecated,NaN,245919,BAYC History -The Great Floor Sweep,SWEEP,18.0,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba
19973358,2,0x26384f4968c166fc110f0ad4ce241c115913244aa003...,0xd9cfa54cc68ee6b67fb35b64e3a452c9c423e38f,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba,1.955085e+06,False,True,2021-08-22 18:32:00,19,152785.0,...,122578,0.004535,deprecated,NaN,1332380,BAYC History -The Great Floor Sweep,SWEEP,18.0,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3a06dd8bf1dc8679113a9ba95e87f4615973afba


In [184]:
EL_ERCneighb_normal = pd.read_csv("erc_first_neighboars_transactions.csv", header = 0)
EL_ERCneighb_normal.head()
random.seed(30)
EL_ERCneib_small = EL_ERCneighb_normal.sample(frac = 0.1) #edge list of neighbor tx (ERC20)

## calc USD value of ERC TX and ERC TX of first neighb

### USD value ERC TX original

In [8]:
erc20_df.timeStamp = erc20_df.timeStamp.astype(str)
df_origin_erc20_USD = pd.merge(erc20_df, ethUSD_s, on='timeStamp')
df_origin_erc20_USD["eth_valueUSD"] = df_origin_erc20_USD.eth_value.astype("float") * df_origin_erc20_USD.Close
df_origin_erc20_USD["transaction_costUSD"] = df_origin_erc20_USD.transaction_cost.astype("float") * df_origin_erc20_USD.Close
df_origin_erc20_USD["will_gas_PriceUSD"] = df_origin_erc20_USD.will_gas_Price.astype("float") * df_origin_erc20_USD.Close

In [23]:
erc20_df

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,will_gas_Price,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed,tokenName,tokenSymbol,tokenDecimal
0,0,0x4558611a8ac6a70eacf40905dc6ed66a2d546e19d37a...,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,1.690967e+05,False,True,2018-06-19 22:49:00,453668,410000.0,8.000000e-09,108066,0.000865,deprecated,NaN,2963337,Pundi X Token,NPXS,18
1,1,0x66e267f021407d5d348825f00fab3a508d40ba5bbbcb...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1.000000e+03,True,True,2018-06-20 06:39:00,12,52909.0,4.100000e-08,52909,0.002169,deprecated,NaN,1924647,Pundi X Token,NPXS,18
2,2,0xa365ef5b0ec87c8b577218d558626ef566adecfc8d55...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1.680967e+05,True,True,2018-06-20 06:46:00,13,38101.0,4.100000e-08,23101,0.000947,deprecated,NaN,1802212,Pundi X Token,NPXS,18
3,3,0xe66fd18b34afacd075abc13e8919d126d1ffd5a23740...,0x79cee3a98afc7f46436e71b452751b8b936b7129,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,8.587814e+04,False,True,2018-07-18 02:56:00,561,300000.0,3.000000e-09,63432,0.000190,deprecated,NaN,7434623,SKYFchain,SKYFT,18
4,4,0x2afac0a99fb3214c71649f41a26a8355ce92ec6e16d6...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x3072d320d538919775e3c8feea6b2fb5721d2693,8.587814e+04,True,True,2018-08-17 09:34:00,15,54612.0,4.100000e-08,39612,0.001624,deprecated,NaN,5084140,SKYFchain,SKYFT,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64717,64717,0x45f4c5a77a11d575a8c70dcc0c8ea148bf0694de0b0c...,0xa5025faba6e70b84f74e9b1113e5f7f4e7f4859f,0x2fe4b878e1fd34e7ec449e7521b8333495608805,4.918000e-10,False,True,2021-09-02 11:37:00,2,1144055.0,7.630133e-08,807281,0.061597,deprecated,NaN,3552874,Tether USD,USDT,6
64718,64718,0x45f4c5a77a11d575a8c70dcc0c8ea148bf0694de0b0c...,0xa5025faba6e70b84f74e9b1113e5f7f4e7f4859f,0x2fe4b878e1fd34e7ec449e7521b8333495608805,4.918000e-10,False,True,2021-09-02 11:37:00,2,1144055.0,7.630133e-08,807281,0.061597,deprecated,NaN,3552874,Tether USD,USDT,6
64719,64719,0x45f4c5a77a11d575a8c70dcc0c8ea148bf0694de0b0c...,0xa5025faba6e70b84f74e9b1113e5f7f4e7f4859f,0x2fe4b878e1fd34e7ec449e7521b8333495608805,4.918000e-10,False,True,2021-09-02 11:37:00,2,1144055.0,7.630133e-08,807281,0.061597,deprecated,NaN,3552874,Tether USD,USDT,6
64720,64720,0xfe6eea3c3def2bcffcf921ad82994e1a4193b3c58611...,0x74de5d4fcbf63e00296fd95d33236b9794016631,0x2fe4b878e1fd34e7ec449e7521b8333495608805,5.050500e-14,False,True,2021-09-02 15:44:00,1397,224692.0,1.670747e-07,173225,0.028942,deprecated,NaN,6539733,AtariToken,ATRI,0


In [9]:
df_origin_erc20_USD

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,...,data_transaction,isError,cumulativeGasUsed,tokenName,tokenSymbol,tokenDecimal,Close,eth_valueUSD,transaction_costUSD,will_gas_PriceUSD
0,0,0x4558611a8ac6a70eacf40905dc6ed66a2d546e19d37a...,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,169096.672827,False,True,2018-06-19 22:49:00,453668,410000.0,...,deprecated,NaN,2963337,Pundi X Token,NPXS,18,521.50,8.818391e+07,0.450851,0.000004
1,1,0x66e267f021407d5d348825f00fab3a508d40ba5bbbcb...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1000.000000,True,True,2018-06-20 06:39:00,12,52909.0,...,deprecated,NaN,1924647,Pundi X Token,NPXS,18,521.42,5.214200e+05,1.131100,0.000021
2,2,0xa365ef5b0ec87c8b577218d558626ef566adecfc8d55...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,168096.672827,True,True,2018-06-20 06:46:00,13,38101.0,...,deprecated,NaN,1802212,Pundi X Token,NPXS,18,520.66,8.752121e+07,0.493138,0.000021
3,3,0xe66fd18b34afacd075abc13e8919d126d1ffd5a23740...,0x79cee3a98afc7f46436e71b452751b8b936b7129,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,85878.140364,False,True,2018-07-18 02:56:00,561,300000.0,...,deprecated,NaN,7434623,SKYFchain,SKYFT,18,502.85,4.318382e+07,0.095690,0.000002
4,4,0x2afac0a99fb3214c71649f41a26a8355ce92ec6e16d6...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x3072d320d538919775e3c8feea6b2fb5721d2693,85878.140364,True,True,2018-08-17 09:34:00,15,54612.0,...,deprecated,NaN,5084140,SKYFchain,SKYFT,18,299.58,2.572737e+07,0.486545,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55815,64600,0xcd35b01459136e11a88dd43483904d4ec4ea0671c3e8...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74,1543.341928,True,True,2021-07-14 00:39:00,990,167032.0,...,deprecated,NaN,6322438,SAND,SAND,18,1912.30,2.951333e+06,5.714641,0.000057
55816,64601,0xd82c0a5abed1c67c02294bd624727b2dd8398f08ad57...,0xd152f549545093347a162dce210e7293f1452150,0x2fe4b878e1fd34e7ec449e7521b8333495608805,97.350000,False,True,2021-07-14 21:12:00,343,735785.0,...,deprecated,NaN,10680966,OCC,OCC,18,1990.73,1.937976e+05,44.167668,0.000062
55817,64602,0xff47934465b8d614da2fc950e317bcf66a8ca9e0fdba...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xebc86fb12ab0ffac6cbcafce2f049bfe7efada0d,97.350000,True,True,2021-07-14 21:23:00,993,200000.0,...,deprecated,NaN,3738598,OCC,OCC,18,1991.11,1.938346e+05,6.761937,0.000056
55818,64603,0xb09ac8e9c51e39e89102268e06101f4da2701013a8a8...,0x84064d4f59353627be67bd28c27ed45517d64d2e,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0.040000,False,True,2021-07-15 18:35:00,995,266421.0,...,deprecated,NaN,2465627,Wrapped Ether,WETH,18,1910.33,7.641320e+01,20.030204,0.000082


### USD value ERC TX neighb

In [97]:
EL_ERCneighb_normal.timeStamp = EL_ERCneighb_normal.timeStamp.astype(str)
EL_ERCneighb_USD = pd.merge(EL_ERCneighb_normal, ethUSD_s, on='timeStamp')
EL_ERCneighb_USD["eth_valueUSD"] = EL_ERCneighb_USD.eth_value.astype("float") * EL_ERCneighb_USD.Close
EL_ERCneighb_USD["transaction_costUSD"] = EL_ERCneighb_USD.transaction_cost.astype("float") * EL_ERCneighb_USD.Close
EL_ERCneighb_USD["will_gas_PriceUSD"] = EL_ERCneighb_USD.will_gas_Price.astype("float") * EL_ERCneighb_USD.Close

In [185]:
import random
random.seed(30)
EL_ERCneibUSD_small = EL_ERCneighb_USD.sample(frac = 0.1)

# Start Variable creation 



## Variables regarding the "normal transactions" of the original addresses

In [37]:
#Number ingoing transactions
length2=[]
for address in df3.wallet_address:
    length2.append(len(df_test[df_test["tx_to"] == str(address)]))
df3.loc[:,"num_in_trans"]= length2

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [102]:
#Number ingoing transactions
length2=[]
for address in df3.wallet_address:
    length2.append(len(df_test[df_test["tx_to"] == str(address)]))
df3.loc[:,"num_in_trans"]= length2

In [38]:
#Number outgoing transactions
length1=[]
for address in df3.wallet_address:
    length1.append(len(df_test[df_test["tx_from"] == str(address)]))
df3.loc[:,"num_out_trans"]= length1

In [104]:
#average amount outgoing transactions
avg_trans_out=[]
for address in df3.wallet_address:
    avg_trans_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["eth_valueUSD"]].mean())
df3.loc[:,"avg_trans_out"]= avg_trans_out

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [40]:
df3.to_csv("corrected_numoutsins.csv")

In [ ]:
df_test

In [105]:
#average amount ingoing transactions
avg_trans_in=[]
for address in df3.wallet_address:
    avg_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].mean())
df3.loc[:,"avg_trans_in"]= avg_trans_in

In [106]:
#sum amount ingoing transactions
sum_trans_in=[]
for address in df3.wallet_address:
    sum_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].sum())
df3.loc[:,"sum_trans_in"]= sum_trans_in

In [107]:
#sum amount outgoing transactions
sum_trans_out=[]
for address in df3.wallet_address:
    sum_trans_out.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].sum())
df3.loc[:,"sum_trans_out"]= sum_trans_out

In [108]:
#max amount outgoing transaction
max_trans_out=[]
for address in df3.wallet_address:
    max_trans_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["eth_valueUSD"]].max())
df3.loc[:,"max_trans_out"]= max_trans_out

In [109]:
#max amount ingoing transaction
max_trans_in=[]
for address in df3.wallet_address:
    max_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].max())
df3.loc[:,"max_trans_in"]= max_trans_in

In [110]:
#standard deviation amount ingoing transaction
std_trans_in=[]
for address in df3.wallet_address:
    std_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].std())
df3.loc[:,"std_trans_in"]= std_trans_in

In [111]:
#standard deviation amount out going transaction
std_trans_out=[]
for address in df3.wallet_address:
    std_trans_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["eth_valueUSD"]].std())
df3.loc[:,"std_trans_out"]= std_trans_out

In [112]:
#min amount outgoing transaction
min_trans_out=[]
for address in df3.wallet_address:
    min_trans_out.append(df_test.loc[(df_test['tx_from'] == address) & (df_test["eth_valueUSD"]!=0),["eth_valueUSD"]].min())
df3.loc[:,"min_trans_out"]= min_trans_out

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [113]:
#min amount ingoing transaction
min_trans_in=[]
for address in df3.wallet_address:
    min_trans_in.append(df_test.loc[(df_test['tx_to'] == address) & (df_test["eth_valueUSD"]!=0),["eth_valueUSD"]].min())
df3.loc[:,"min_trans_in"]= min_trans_in

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [114]:
#number of unique outgoing transaction partners 
unique_tpartn_out=[]
for address in df3.wallet_address:
    unique_tpartn_out.append(df_test.loc[(df_test['tx_from'] == address), ["tx_to"]].nunique())
df3.loc[:,"unique_tpartn_out"]= unique_tpartn_out


In [115]:
#number of unique ingoing transaction partners 
unique_tpartn_in=[]
for address in df3.wallet_address:
    unique_tpartn_in.append(df_test.loc[(df_test['tx_to'] == address), ["tx_from"]].nunique())
df3.loc[:,"unique_tpartn_in"]= unique_tpartn_in

In [116]:
#average outg. transaction costs
avg_transcost_out=[]
for address in df3.wallet_address:
    avg_transcost_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"avg_transcost_out"]= avg_transcost_out 

In [117]:
#average inc. transaction costs
avg_transcost_in=[]
for address in df3.wallet_address:
    avg_transcost_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"avg_transcost_in"]= avg_transcost_in

In [118]:
#max transaction costs outgoing
max_transcost_out=[]
for address in df3.wallet_address:
    max_transcost_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["transaction_costUSD"]].max())
df3.loc[:,"max_transcost_out"]= max_transcost_out

In [119]:
#max transaction costs ingoing
max_transcost_in=[]
for address in df3.wallet_address:
    max_transcost_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["transaction_costUSD"]].max())
df3.loc[:,"max_transcost_in"]= max_transcost_in

In [120]:
#standard deviation transaction costs  out going 
std_transcost_out=[]
for address in df3.wallet_address:
    std_transcost_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["transaction_costUSD"]].std())
df3.loc[:,"std_transcost_out"]= std_transcost_out

In [121]:
#standard deviation transaction costs  ingoing 
std_transcost_in=[]
for address in df3.wallet_address:
    std_transcost_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["transaction_costUSD"]].std())
df3.loc[:,"std_transcost_in"]= std_transcost_in

In [122]:
#avg. amount of hours between outg transactions
timedelta_trans_out=[]
for address in df3.wallet_address:
    t = pd.to_datetime(df_test.loc[lambda df_test: df_test['tx_from'] == address, ["timeStamp"]].timeStamp)
    time_diff = [j-i for i, j in zip(t[:-1], t[1:])]
    time_list = []
    for delta in time_diff:
        time_list.append(delta / pd.Timedelta('1 hour'))
    timedelta_trans_out.append(pd.Series(time_list).mean())
    
df3.loc[:,"timedelta_trans_out"]= timedelta_trans_out


<ipython-input-122-62aaec5a58bd>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  timedelta_trans_out.append(pd.Series(time_list).mean())
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [123]:
#avg. amount of hours between in transactions
timedelta_trans_in=[]
for address in df3.wallet_address:
    t = pd.to_datetime(df_test.loc[lambda df_test: df_test['tx_to'] == address, ["timeStamp"]].timeStamp)
    time_diff = [j-i for i, j in zip(t[:-1], t[1:])]
    time_list = []
    for delta in time_diff:
        time_list.append(delta / pd.Timedelta('1 hour'))
    timedelta_trans_in.append(pd.Series(time_list).mean())
    
df3.loc[:,"timedelta_trans_in"]= timedelta_trans_in

<ipython-input-123-d986470869da>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  timedelta_trans_in.append(pd.Series(time_list).mean())


In [124]:
#amount of hours between the first and last outgoing transaction
duration_out=[]
for address in df3.wallet_address:
    t = pd.to_datetime(df_test.loc[lambda df_test: df_test['tx_from'] == address, ["timeStamp"]].timeStamp)
    time_diff = [j-i for i, j in zip(t[:1], t[-1:])]
    time_list = []
    for delta in time_diff:
        time_list.append(delta / pd.Timedelta('1 hour'))
    duration_out.append(pd.Series(time_list).mean())
    
df3.loc[:,"duration_out"]= duration_out

<ipython-input-124-2cfee261268d>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  duration_out.append(pd.Series(time_list).mean())


In [125]:
#avg Number ingoing transactions per neighb
length_neighb=[]
for address in df3.wallet_address:
    result = 0
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars.loc[(neighboars["id"] == str(address))]
    length_p_neighb = []
    for index, row in filtered_neighb.iterrows():
        length_p_neighb.append(len(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar)]))
        result = pd.DataFrame(length_p_neighb)
        result = result.iloc[:,0].mean()
    length_neighb.append(result)
        
df3.loc[:,"num_in_trans_1neighb"]= length_neighb



/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [126]:
#percentage of transaction partners who received only one transaction from the address (sog. one Night Stands
avg_oneNightStands_out = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        p_address = filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["neighboar"]].count()
        result_p_address.append(p_address)
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].eq(1).sum(axis = 0)
        result2 = result
        counts = filtered_neighb.iloc[:,1].count()
        result3 = result2/counts
    avg_oneNightStands_out.append(result3)
        
df3.loc[:,"avg_oneNightStands_out"]= avg_oneNightStands_out

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [127]:
#percentage of transaction partners who received only one transaction from the address (sog. one Night Stands
avg_oneNightStands_in = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        p_address = filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["neighboar"]].count()
        result_p_address.append(p_address)
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].eq(1).sum(axis = 0)
        result2 = result
        counts = filtered_neighb.iloc[:,1].count()
        result3 = result2/counts
    avg_oneNightStands_in.append(result3)
        
df3.loc[:,"avg_oneNightStands_in"]= avg_oneNightStands_in

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
filtered_neighb.iloc[:,1].count()

## variables regarding the normal transactions of the first neighbs 

In [130]:
#avg Number ingoing transactions per neighb
length_neighb=[]
for address in df3.wallet_address:
    result = 0
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_["id"] == str(address))]
    length_p_neighb = []
    for index, row in filtered_neighb.iterrows():
        length_p_neighb.append(len(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar)]))
        result = pd.DataFrame(length_p_neighb)
        result = result.iloc[:,0].mean()
    length_neighb.append(result)
        
df3.loc[:,"num_in_trans_1neighb"]= length_neighb


NameError: name 'neighboars_' is not defined

In [ ]:
#avg Number outgoing transactions per neighb
length_neighb_out=[]
for address in df3.wallet_address:
    result = 0
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    length_p_neighb = []
    for index, row in filtered_neighb.iterrows():
        length_p_neighb.append(len(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar)]))
        result = pd.DataFrame(length_p_neighb)
        result = result.iloc[:,0].mean()
    length_neighb_out.append(result)
        
df3.loc[:,"num_out_trans_1neighb"]= length_neighb_out



In [ ]:
#average amount outgoing transactions p neighb
avg_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_tx_out_neighb.append(result)
        
df3.loc[:,"avg_tx_out_neighb"]= avg_tx_out_neighb

In [ ]:
#average amount ingoing transactions p neighb
avg_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_tx_in_neighb.append(result)
        
df3.loc[:,"avg_tx_in_neighb"]= avg_tx_in_neighb

In [131]:
#max amount outgoing transactions 
max_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_tx_out_neighb.append(result)
        
df3.loc[:,"max_tx_out_neighb"]= max_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [132]:
#max amount ingoing transactions 
max_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_tx_in_neighb.append(result)
        
df3.loc[:,"max_tx_in_neighb"]= max_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [133]:
#sum amount ingoing transactions 
sum_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].sum())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    sum_tx_in_neighb.append(result)
        
df3.loc[:,"sum_tx_in_neighb"]= sum_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [134]:
#sum amount outgoing transactions 
sum_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].sum())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    sum_tx_out_neighb.append(result)
        
df3.loc[:,"sum_tx_out_neighb"]= sum_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [135]:
#standard deviation amount outgoing transaction
std_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].std())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    std_tx_out_neighb.append(result)
        
df3.loc[:,"std_tx_out_neighb"]= std_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [136]:
#standard deviation amount ingoing transaction
std_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].std())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    std_tx_in_neighb.append(result)
        
df3.loc[:,"std_tx_in_neighb"]= std_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [137]:
#number of unique outgoing transaction partners 
unique_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["tx_to"]].nunique().iloc[0])
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    unique_tx_out_neighb.append(result)
        
df3.loc[:,"unique_tx_out_neighb"]= unique_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [138]:
#number of unique ingoing transaction partners 
unique_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["tx_to"]].nunique().iloc[0])
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    unique_tx_in_neighb.append(result)
        
df3.loc[:,"unique_tx_in_neighb"]= unique_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [139]:
#average amount outgoing transaction cost 
avg_txcost_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_txcost_out_neighb.append(result)
        
df3.loc[:,"avg_txcost_out_neighb"]= avg_txcost_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [140]:
#average amount ingoing transaction cost 
avg_txcost_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_txcost_in_neighb.append(result)
        
df3.loc[:,"avg_txcost_in_neighb"]= avg_txcost_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [141]:
#max amount ingoing transaction cost
max_txcost_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_txcost_in_neighb.append(result)
        
df3.loc[:,"max_txcost_in_neighb"]= max_txcost_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [142]:
#max amount out going transaction cost
max_txcost_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_txcost_out_neighb.append(result)
        
df3.loc[:,"max_txcost_out_neighb"]= max_txcost_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [143]:
#avg balance per neighb
avg_balance_p_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)&(df_Neighb_normal_USD["transaction_costUSD"] != 0)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        sleep(1)
        result_p_address.append(int(balance_api_call(str(row.neighboar), "QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ")))
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_balance_p_neighb.append(result)
        
df3.loc[:,"avg_balance_p_neighb"]= avg_balance_p_neighb

TypeError: int() argument must be a string, a bytes-like object or a number, not 'dict'

In [148]:
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)&(df_Neighb_normal_USD["transaction_costUSD"] != 0)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        sleep(1)
        print(row.neighboar, int(balance_api_call(str(row.neighboar), "QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ")))

0xb1690c08e213a35ed9bab7b318de14420fb57d8c 251611470729359269046
0x06012c8cf97bead5deae237070f9587f8e7a266d 6978962596193186753
0x4d39217fd94abf03928da97488b16ac91816108f 12950000000000000
0x199f2202d878e9747b4f96cdf2c980d59d7c0969 10000000000000000
0xdd4678e7f3d7ee45c62a0e3424a7a95d82a86489 1
0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208 18075258604266470990491
0xa15c7ebe1f07caf6bff097d8a589fb8ac49ae5b3 0
0x5dd0815a4cf119ad91ba045bbbf879f3f7de3c68 0
0xcb47c89cb17c10b719fc5ed9665bae157cac2cb1 76910895114692639733
0x946048a75af11c300a274344887ec39452218b3d 0
0x41ec0ddb4e3a17e9b0a6bae7d90f2d389aaf7dee 810000000000000
0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359 0
0x6758b7d441a9739b98552b373703d8d3d14f9e62 0
0x4e9d7f37eadc6fef64b5f5dccc4deb6224667677 0
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0
0x37e02a58cbbb2bfd8bb097517b3bbda0e4604eaa 0
0xf45277b9b338a83b39d74419c510a6077d8c7734 174003577124655
0x34a34267ec4034ba52f154829d0b6548bbbf7c74 0
0x9677e98c621868a97d2dc0678efa7203475e9981 0
0xd46b

KeyboardInterrupt: 

In [149]:
df3.to_csv("first_data.csv")



## Get ERC variables for origin addresses

##

In [4]:
df3 = pd.read_csv("first_data.csv")

In [19]:
print(df_origin_erc20_USD.hash.iloc[0])

0x4558611a8ac6a70eacf40905dc6ed66a2d546e19d37a893620105e1bc33eaf51


In [22]:
df_origin_erc20_USD

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,...,data_transaction,isError,cumulativeGasUsed,tokenName,tokenSymbol,tokenDecimal,Close,eth_valueUSD,transaction_costUSD,will_gas_PriceUSD
0,0,0x4558611a8ac6a70eacf40905dc6ed66a2d546e19d37a...,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,169096.672827,False,True,2018-06-19 22:49:00,453668,410000.0,...,deprecated,NaN,2963337,Pundi X Token,NPXS,18,521.50,8.818391e+07,0.450851,0.000004
1,1,0x66e267f021407d5d348825f00fab3a508d40ba5bbbcb...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1000.000000,True,True,2018-06-20 06:39:00,12,52909.0,...,deprecated,NaN,1924647,Pundi X Token,NPXS,18,521.42,5.214200e+05,1.131100,0.000021
2,2,0xa365ef5b0ec87c8b577218d558626ef566adecfc8d55...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,168096.672827,True,True,2018-06-20 06:46:00,13,38101.0,...,deprecated,NaN,1802212,Pundi X Token,NPXS,18,520.66,8.752121e+07,0.493138,0.000021
3,3,0xe66fd18b34afacd075abc13e8919d126d1ffd5a23740...,0x79cee3a98afc7f46436e71b452751b8b936b7129,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,85878.140364,False,True,2018-07-18 02:56:00,561,300000.0,...,deprecated,NaN,7434623,SKYFchain,SKYFT,18,502.85,4.318382e+07,0.095690,0.000002
4,4,0x2afac0a99fb3214c71649f41a26a8355ce92ec6e16d6...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x3072d320d538919775e3c8feea6b2fb5721d2693,85878.140364,True,True,2018-08-17 09:34:00,15,54612.0,...,deprecated,NaN,5084140,SKYFchain,SKYFT,18,299.58,2.572737e+07,0.486545,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55815,64600,0xcd35b01459136e11a88dd43483904d4ec4ea0671c3e8...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74,1543.341928,True,True,2021-07-14 00:39:00,990,167032.0,...,deprecated,NaN,6322438,SAND,SAND,18,1912.30,2.951333e+06,5.714641,0.000057
55816,64601,0xd82c0a5abed1c67c02294bd624727b2dd8398f08ad57...,0xd152f549545093347a162dce210e7293f1452150,0x2fe4b878e1fd34e7ec449e7521b8333495608805,97.350000,False,True,2021-07-14 21:12:00,343,735785.0,...,deprecated,NaN,10680966,OCC,OCC,18,1990.73,1.937976e+05,44.167668,0.000062
55817,64602,0xff47934465b8d614da2fc950e317bcf66a8ca9e0fdba...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xebc86fb12ab0ffac6cbcafce2f049bfe7efada0d,97.350000,True,True,2021-07-14 21:23:00,993,200000.0,...,deprecated,NaN,3738598,OCC,OCC,18,1991.11,1.938346e+05,6.761937,0.000056
55818,64603,0xb09ac8e9c51e39e89102268e06101f4da2701013a8a8...,0x84064d4f59353627be67bd28c27ed45517d64d2e,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0.040000,False,True,2021-07-15 18:35:00,995,266421.0,...,deprecated,NaN,2465627,Wrapped Ether,WETH,18,1910.33,7.641320e+01,20.030204,0.000082


In [11]:
df_origin_erc20_USD['tokenSymbol'] = pd.Categorical(df_origin_erc20_USD['tokenSymbol'])

In [12]:
print (df_origin_erc20_USD['tokenSymbol'].cat.categories)

Index(['$ANRX', '$BASED', '$BREAST', '$DG', '$FDD', '$ITG', '$JINDOINU',
       '0xBTC', '0xBUTT', '1-UP',
       ...
       'yvWETH', 'yvYFI', 'yveCRV-DAO', 'yyDAI+yUSDC+yUSDT+yTUSD', 'zSK',
       'ziot', 'Ͼ', '⚗️', '✌(◕‿-)✌  tinyurl.com/nicetokens', '✨'],
      dtype='object', length=2128)


In [13]:
#Number ingoing transactions
lengthERC=[]
for address in df3.wallet_address:
    lengthERC.append(len(df_origin_erc20_USD[df_origin_erc20_USD["tx_to"] == str(address)]))
df3.loc[:,"num_in_transERC"]= lengthERC

In [14]:
#Number ingoing transactions
lengthERC=[]
for address in df3.wallet_address:
    lengthERC.append(len(df_origin_erc20_USD[df_origin_erc20_USD["tx_from"] == str(address)]))
df3.loc[:,"num_out_transERC"]= lengthERC

In [ ]:
#average amount outgoing transactions macht kein Sinn, da es insg. über 2100 unterschiedliche Tokens sind. Der Blockexplorer gibt die absolute Anzahl an Tokens aus aber nicht den Preis. Die Preise von 2100 Tokens können nicht herausgesucht werden. Die CoinMarket Cap API bietet das nur für 500 Euro pro Monat an

In [25]:
#number of unique outgoing transaction partners 
ERCunique_tpartn_out=[]
for address in df3.wallet_address:
    ERCunique_tpartn_out.append(df_origin_erc20_USD.loc[(df_origin_erc20_USD['tx_from'] == address), ["tx_to"]].nunique())
df3.loc[:,"ERCunique_tpartn_out"]= ERCunique_tpartn_out

In [26]:
#number of unique ingoing transaction partners 
ERCunique_tpartn_in=[]
for address in df3.wallet_address:
    ERCunique_tpartn_in.append(df_origin_erc20_USD.loc[(df_origin_erc20_USD['tx_to'] == address), ["tx_from"]].nunique())
df3.loc[:,"ERCunique_tpartn_in"]= ERCunique_tpartn_in

In [27]:
#average outg. transaction costs
ERCavg_transcost_out=[]
for address in df3.wallet_address:
    ERCavg_transcost_out.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_from'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"ERCavg_transcost_out"]= ERCavg_transcost_out 

In [68]:
#average inc. transaction costs
ERCavg_transcost_in=[]
for address in df3.wallet_address:
    ERCavg_transcost_in.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_to'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"ERCavg_transcost_in"]= ERCavg_transcost_in

In [69]:
#max transaction costs outgoing
ERCmax_transcost_out=[]
for address in df3.wallet_address:
    ERCmax_transcost_out.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_from'] == str(address), ["transaction_costUSD"]].max())
df3.loc[:,"ERCmax_transcost_out"]= ERCmax_transcost_out

In [108]:
#number of different traded tokens (outgoing + ingoing)
ERC_nb_tkn_out=[]
ERC_nb_tkn_in=[]
ERC_nb_tkn_tot=[]
for address in df3.wallet_address:
    ERC_nb_tkn_out.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_from'] == str(address), ["tokenSymbol"]].nunique().iloc[0])
    ERC_nb_tkn_in.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_to'] == str(address), ["tokenSymbol"]].nunique().iloc[0])
ERC_nb_tkn_tot = pd.DataFrame(ERC_nb_tkn_out)+ pd.DataFrame(ERC_nb_tkn_in)
#df3.loc[:,"ERC_nb_tkn_tot"]= ERC_nb_tkn_tot

In [116]:
print(ERC_nb_tkn_tot.isna().sum())

ERC_nb_tkn_tot    0
dtype: int64


In [98]:
df3 = df3.drop(['ERC_nb_tkn_tot'], axis=1)

In [109]:
ERC_nb_tkn_tot = ERC_nb_tkn_tot.rename({0: 'ERC_nb_tkn_tot'}, axis = 1)

In [118]:
df3 = pd.concat([df3, ERC_nb_tkn_tot], axis = 1)
df3

,Unnamed: 0,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length,hash,address_beginning,address_balance,num_in_trans,...,max_txcost_in_neighb,max_txcost_out_neighb,num_in_transERC,num_out_transERC,ERCunique_tpartn_out,ERCunique_tpartn_in,ERCavg_transcost_out,ERCavg_transcost_in,ERCmax_transcost_out,ERC_nb_tkn_tot
0,15,0.0,mizusize@gmail.com,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,AKIHIRO MIZUGUCHI,42,-4088783140720925316,0x,246167951093634996,76,...,2.076727,27.054047,609,591,170,185,15.146850,22.514325,259.066805,241
1,16,0.0,carjean02@gmail.com,0x090dcf87443e1495b0ffaf1f3b7a1d8b2c91aa3b,Jeancarlos Acosta Mejia,42,7126748193056654943,0x,359507000000000,0,...,0.000000,0.000000,0,0,0,0,NaN,NaN,NaN,0
2,17,0.0,s.d.tolpa@web.de,0xaa9d5a6cdb9344fd22cda14e5de5c1de9c41cc91,SASCHA MRONGOWIUS,42,4719250612253798252,0x,246845980595768771,3,...,0.005313,3.445493,2,1,1,1,3.445493,1.165070,3.445493,2
3,18,0.0,cde@posteo.net,0x9329c22f63898e18e8a67c582fe8092203b90c64,CLAUDIO SEBASTIAN WECK,42,2045020602619597288,0x,5214635318886014489,5,...,0.176530,0.092049,11,1,1,9,4.609975,4.558068,4.609975,9
4,19,0.0,crisgti580@gmail.com,0xbefb4fecd83185d358b891f12eda1c20fdba9c20,IONUT CRISTIAN MIHAIL,42,-2058215953495933286,0x,454540735692808878,23,...,1.554172,28.781189,148,133,43,59,9.468765,7.778343,98.525757,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,349,1.0,shahsaumil@gmail.com,0x0230c0f62a61d77c62c48b162f6dec2be8dc4d20,SAUMIL G SHAH,42,1231488335706197333,0x,NaN,6,...,5.970831,NaN,30,12,6,13,7.260738,11.712182,14.373036,18
299,350,0.0,b_bordenave@live.com,0x06f62b8b2eebde73dd4f0e1fa9b1321d63fd5c3a,Benjamin Bordenave,42,1827549469105083427,0x,NaN,13,...,10.241713,40.577530,29,21,8,9,12.646482,22.980449,56.148393,12
300,352,0.0,ltmemories@gmail.com,0xd5abc758d753b68985b2bb83f2b2f4030179ea20,John Gravagna,42,-3082015307768915211,0x,NaN,0,...,0.000000,0.000000,0,0,0,0,NaN,NaN,NaN,0
301,353,0.0,seleznyov@gmail.com,0x0cead33eb6adf4860b6b8f66687f51567e111973,ANDRII SELEZNOV,42,7925451324816931321,0x,NaN,2,...,1.570387,NaN,13,5,5,9,5.982765,231.186288,16.922603,11


In [127]:
#
df_test

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,will_gas_Price,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed,Close,eth_valueUSD,transaction_costUSD,will_gas_PriceUSD
0,0,0x9255fc5f3aebc183fad66c2cdc8723725b58bc8fdae4...,0x003e36550908907c2a2da960fd19a419b9a774b7,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0.004194,False,True,2018-05-15 08:09:00,48351,21000.0,1.000000e-08,21000,0.000210,0x,0,418019,733.93,3.077756,0.154125,0.000007
1,1,0xdf5ccd2c08ecd7bc8c07fdb936ba183e06fa2a000a40...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.001941,True,True,2018-05-15 08:21:00,0,158820.0,1.000000e-08,60508,0.000605,0x454a2ab3000000000000000000000000000000000000...,0,1889733,734.90,1.426545,0.444673,0.000007
2,2,0x743f000df6bbd13a48fcc02d84adf26fd2a8bc84666b...,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0.040000,False,True,2018-05-17 07:11:00,2362530,21000.0,5.000000e-08,21000,0.001050,0x,0,3711616,701.35,28.054309,0.736418,0.000035
3,3,0xb1abcc5b22c2f1fee9d1591a7060842e44b14586a385...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,0.002333,True,True,2018-05-17 07:58:00,1,135963.0,1.000000e-08,45508,0.000455,0x454a2ab3000000000000000000000000000000000000...,0,7476882,699.93,1.633035,0.318524,0.000007
4,4,0x685ff133d98dd4da61bc0ac7e85c52d4aa7b3debd1b3...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x06012c8cf97bead5deae237070f9587f8e7a266d,0.008000,True,True,2018-05-17 08:15:00,2,119977.0,1.000000e-08,79985,0.000800,0xf7d8c883000000000000000000000000000000000000...,0,3803770,701.15,5.609200,0.560815,0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64636,64636,0x8af603cf08160290dd98a9ccc98d31aa916275bc8892...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0.100000,True,True,2021-07-14 09:53:00,992,218853.0,4.000000e-08,208945,0.008358,0xab834bab0000000000000000000000007be8076f4ea4...,0,11442929,1932.79,193.279000,16.153872,0.000077
64637,64637,0xff47934465b8d614da2fc950e317bcf66a8ca9e0fdba...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xebc86fb12ab0ffac6cbcafce2f049bfe7efada0d,0.000000,True,True,2021-07-14 21:23:00,993,200000.0,2.800000e-08,121288,0.003396,0x1bf6ddae000000000000000000000000000000000000...,0,3738598,1991.11,0.000000,6.761937,0.000056
64638,64638,0x42e548c91e06188c3e093db7650b2f665dd945aab37a...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0.000000,True,True,2021-07-15 18:30:00,994,46280.0,4.150000e-08,46280,0.001921,0xa22cb465000000000000000000000000239155fb141a...,0,14614544,1911.14,0.000000,3.670574,0.000079
64639,64639,0xb09ac8e9c51e39e89102268e06101f4da2701013a8a8...,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0.000000,True,True,2021-07-15 18:35:00,995,266421.0,4.300000e-08,243842,0.010485,0xab834bab0000000000000000000000007be8076f4ea4...,0,2465627,1910.33,0.000000,20.030204,0.000082


In [154]:
df_test["timeStamp"] = pd.to_datetime(df_test["timeStamp"])

In [196]:
#number of sells
hot_sell_per = df_test[df_test["out"]==True].resample("H", on = "timeStamp").out.count()
hot_sell_per

timeStamp
2017-05-01 21:00:00     1
2017-05-01 22:00:00     0
2017-05-01 23:00:00     0
2017-05-02 00:00:00     0
2017-05-02 01:00:00     0
                       ..
2021-07-16 19:00:00     5
2021-07-16 20:00:00     3
2021-07-16 21:00:00    12
2021-07-16 22:00:00    10
2021-07-16 23:00:00    17
Freq: H, Name: out, Length: 36891, dtype: int64

In [244]:
#number of buys
hot_buy_per = df_test[df_test["out"]==False].resample("D", on = "timeStamp").hash.count()
hot_buy_per

timeStamp
2017-05-01     3
2017-05-02     0
2017-05-03     0
2017-05-04     0
2017-05-05     0
              ..
2021-07-12     7
2021-07-13     0
2021-07-14    11
2021-07-15    12
2021-07-16    11
Freq: D, Name: hash, Length: 1538, dtype: int64

In [245]:
pressure_periods = pd.concat([hot_buy_per, hot_sell_per], axis = 1, join = "inner")
pressure_periods = pressure_periods.rename({"hash":"no_buys", "out":"no_sells"}, axis = "columns")
pressure_periods["p"]= pressure_periods["no_buys"] - pressure_periods["no_sells"]
pressure_periods[["p"]].describe()

,p
count,1537.000000
mean,3.409239
std,25.193465
min,-16.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,877.000000


In [248]:
sorted_pp = pressure_periods.sort_values(by=["p"])
sorted_pp.head(10)


,no_buys,no_sells,p
timeStamp,,,
2021-04-22,30,46,-16
2020-11-14,0,14,-14
2020-10-29,6,19,-13
2020-11-16,2,13,-11
2020-10-24,5,16,-11
2018-05-05,0,10,-10
2020-12-26,4,13,-9
2020-11-20,5,14,-9
2020-12-10,4,12,-8


In [319]:
sellpump = sorted_pp.head(10)
sellpump

,no_buys,no_sells,p
timeStamp,,,
2021-04-22,30,46,-16
2020-11-14,0,14,-14
2020-10-29,6,19,-13
2020-11-16,2,13,-11
2020-10-24,5,16,-11
2018-05-05,0,10,-10
2020-12-26,4,13,-9
2020-11-20,5,14,-9
2020-12-10,4,12,-8


In [338]:
buypump = sorted_pp.tail(10)
buypump

,no_buys,no_sells,p
timeStamp,,,
2021-05-16,49,4,45
2021-04-27,52,6,46
2021-05-20,58,6,52
2021-05-03,53,1,52
2021-04-03,58,2,56
2021-04-01,144,15,129
2021-03-31,140,9,131
2021-03-30,197,9,188
2021-04-02,294,6,288


In [293]:
buys =[]
eth_bough = []
address_id = []
time_index = []
for address in df3.wallet_address:
    filter_df = df_test.loc[df_test["tx_to"]==str(address)]
    for i in filter_df.resample("D", on = "timeStamp").eth_value.sum().index:
        if i in buypump.index: 
            
            
            buy_df["eth_bought"] = filter_df.resample("D", on = "timeStamp").eth_value.sum()
            buy_df["address"] = address
            buy_df["time"] = i
        #buys.append()

TypeError: list indices must be integers or slices, not str

In [321]:
df3

,Unnamed: 0,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length,hash,address_beginning,address_balance,num_in_trans,...,max_txcost_in_neighb,max_txcost_out_neighb,num_in_transERC,num_out_transERC,ERCunique_tpartn_out,ERCunique_tpartn_in,ERCavg_transcost_out,ERCavg_transcost_in,ERCmax_transcost_out,ERC_nb_tkn_tot
0,15,0.0,mizusize@gmail.com,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,AKIHIRO MIZUGUCHI,42,-4088783140720925316,0x,246167951093634996,76,...,2.076727,27.054047,609,591,170,185,15.146850,22.514325,259.066805,241
1,16,0.0,carjean02@gmail.com,0x090dcf87443e1495b0ffaf1f3b7a1d8b2c91aa3b,Jeancarlos Acosta Mejia,42,7126748193056654943,0x,359507000000000,0,...,0.000000,0.000000,0,0,0,0,NaN,NaN,NaN,0
2,17,0.0,s.d.tolpa@web.de,0xaa9d5a6cdb9344fd22cda14e5de5c1de9c41cc91,SASCHA MRONGOWIUS,42,4719250612253798252,0x,246845980595768771,3,...,0.005313,3.445493,2,1,1,1,3.445493,1.165070,3.445493,2
3,18,0.0,cde@posteo.net,0x9329c22f63898e18e8a67c582fe8092203b90c64,CLAUDIO SEBASTIAN WECK,42,2045020602619597288,0x,5214635318886014489,5,...,0.176530,0.092049,11,1,1,9,4.609975,4.558068,4.609975,9
4,19,0.0,crisgti580@gmail.com,0xbefb4fecd83185d358b891f12eda1c20fdba9c20,IONUT CRISTIAN MIHAIL,42,-2058215953495933286,0x,454540735692808878,23,...,1.554172,28.781189,148,133,43,59,9.468765,7.778343,98.525757,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,349,1.0,shahsaumil@gmail.com,0x0230c0f62a61d77c62c48b162f6dec2be8dc4d20,SAUMIL G SHAH,42,1231488335706197333,0x,NaN,6,...,5.970831,NaN,30,12,6,13,7.260738,11.712182,14.373036,18
299,350,0.0,b_bordenave@live.com,0x06f62b8b2eebde73dd4f0e1fa9b1321d63fd5c3a,Benjamin Bordenave,42,1827549469105083427,0x,NaN,13,...,10.241713,40.577530,29,21,8,9,12.646482,22.980449,56.148393,12
300,352,0.0,ltmemories@gmail.com,0xd5abc758d753b68985b2bb83f2b2f4030179ea20,John Gravagna,42,-3082015307768915211,0x,NaN,0,...,0.000000,0.000000,0,0,0,0,NaN,NaN,NaN,0
301,353,0.0,seleznyov@gmail.com,0x0cead33eb6adf4860b6b8f66687f51567e111973,ANDRII SELEZNOV,42,7925451324816931321,0x,NaN,2,...,1.570387,NaN,13,5,5,9,5.982765,231.186288,16.922603,11


In [328]:
df3_cleaner = df3.loc[df3["num_in_trans"]!=0]

In [329]:
df3_cleaner = df3_cleaner.loc[df3_cleaner["num_out_trans"]!=0]
df3_cleaner

,Unnamed: 0,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length,hash,address_beginning,address_balance,num_in_trans,...,max_txcost_in_neighb,max_txcost_out_neighb,num_in_transERC,num_out_transERC,ERCunique_tpartn_out,ERCunique_tpartn_in,ERCavg_transcost_out,ERCavg_transcost_in,ERCmax_transcost_out,ERC_nb_tkn_tot
0,15,0.0,mizusize@gmail.com,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,AKIHIRO MIZUGUCHI,42,-4088783140720925316,0x,246167951093634996,76,...,2.076727,27.054047,609,591,170,185,15.146850,22.514325,259.066805,241
2,17,0.0,s.d.tolpa@web.de,0xaa9d5a6cdb9344fd22cda14e5de5c1de9c41cc91,SASCHA MRONGOWIUS,42,4719250612253798252,0x,246845980595768771,3,...,0.005313,3.445493,2,1,1,1,3.445493,1.165070,3.445493,2
3,18,0.0,cde@posteo.net,0x9329c22f63898e18e8a67c582fe8092203b90c64,CLAUDIO SEBASTIAN WECK,42,2045020602619597288,0x,5214635318886014489,5,...,0.176530,0.092049,11,1,1,9,4.609975,4.558068,4.609975,9
4,19,0.0,crisgti580@gmail.com,0xbefb4fecd83185d358b891f12eda1c20fdba9c20,IONUT CRISTIAN MIHAIL,42,-2058215953495933286,0x,454540735692808878,23,...,1.554172,28.781189,148,133,43,59,9.468765,7.778343,98.525757,75
5,29,0.0,kelechi.chinonso@gmail.com,0x88c4181152f4133a7a63cabee0c601f75ddeb867,KELECHI ROBERT NWOSU,42,7212562729224340191,0x,225232253829925938,24,...,2.192133,15.306654,24,12,9,17,3.586294,6.145770,16.322677,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,332,0.0,jhonfrerock@gmail.com,0x15ddb1e93b46a60277ae63ffee7defc5500137e5,JHON FREYDER SIACHOQUE CHILATRA,42,-3422320141211135142,0x,NaN,32,...,3.292409,4.190276,24,16,10,9,15.732324,22.988476,44.311866,13
298,349,1.0,shahsaumil@gmail.com,0x0230c0f62a61d77c62c48b162f6dec2be8dc4d20,SAUMIL G SHAH,42,1231488335706197333,0x,NaN,6,...,5.970831,NaN,30,12,6,13,7.260738,11.712182,14.373036,18
299,350,0.0,b_bordenave@live.com,0x06f62b8b2eebde73dd4f0e1fa9b1321d63fd5c3a,Benjamin Bordenave,42,1827549469105083427,0x,NaN,13,...,10.241713,40.577530,29,21,8,9,12.646482,22.980449,56.148393,12
301,353,0.0,seleznyov@gmail.com,0x0cead33eb6adf4860b6b8f66687f51567e111973,ANDRII SELEZNOV,42,7925451324816931321,0x,NaN,2,...,1.570387,NaN,13,5,5,9,5.982765,231.186288,16.922603,11


In [335]:
high_pressure_sells =[]

for address in df3.wallet_address:
    test = pd.DataFrame()
    eth_bough = []
    address_id = []
    time_index = []
    filter_df = df_test.loc[df_test["tx_from"]==str(address)]
    for i in filter_df.resample("D", on = "timeStamp").eth_value.sum().index:
        if i in sellpump.index: 
            filter2 = filter_df[filter_df["timeStamp"]== str(i)]
            address_id.append(address)
            time_index.append(i)
    test["address_id"] = address_id
    test["time_index"] = time_index
    high_pressure_sells.append(len(test))
df3.loc[:,"high_pressure_sells"]= high_pressure_sells
        #buys.append()

In [339]:
high_pressure_buys =[]

for address in df3.wallet_address:
    test = pd.DataFrame()
    eth_bough = []
    address_id = []
    time_index = []
    filter_df = df_test.loc[df_test["tx_from"]==str(address)]
    for i in filter_df.resample("D", on = "timeStamp").eth_value.sum().index:
        if i in buypump.index: 
            filter2 = filter_df[filter_df["timeStamp"]== str(i)]
            address_id.append(address)
            time_index.append(i)
    test["address_id"] = address_id
    test["time_index"] = time_index
    high_pressure_buys.append(len(test))
df3.loc[:,"high_pressure_buys"]= high_pressure_buys
        #buys.append()

In [340]:
df3

,Unnamed: 0,sow_estimated_net_worth_value,address,wallet_address,full_name,address_length,hash,address_beginning,address_balance,num_in_trans,...,num_in_transERC,num_out_transERC,ERCunique_tpartn_out,ERCunique_tpartn_in,ERCavg_transcost_out,ERCavg_transcost_in,ERCmax_transcost_out,ERC_nb_tkn_tot,high_pressure_sells,high_pressure_buys
0,15,0.0,mizusize@gmail.com,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,AKIHIRO MIZUGUCHI,42,-4088783140720925316,0x,246167951093634996,76,...,609,591,170,185,15.146850,22.514325,259.066805,241,9,10
1,16,0.0,carjean02@gmail.com,0x090dcf87443e1495b0ffaf1f3b7a1d8b2c91aa3b,Jeancarlos Acosta Mejia,42,7126748193056654943,0x,359507000000000,0,...,0,0,0,0,NaN,NaN,NaN,0,0,0
2,17,0.0,s.d.tolpa@web.de,0xaa9d5a6cdb9344fd22cda14e5de5c1de9c41cc91,SASCHA MRONGOWIUS,42,4719250612253798252,0x,246845980595768771,3,...,2,1,1,1,3.445493,1.165070,3.445493,2,0,0
3,18,0.0,cde@posteo.net,0x9329c22f63898e18e8a67c582fe8092203b90c64,CLAUDIO SEBASTIAN WECK,42,2045020602619597288,0x,5214635318886014489,5,...,11,1,1,9,4.609975,4.558068,4.609975,9,5,0
4,19,0.0,crisgti580@gmail.com,0xbefb4fecd83185d358b891f12eda1c20fdba9c20,IONUT CRISTIAN MIHAIL,42,-2058215953495933286,0x,454540735692808878,23,...,148,133,43,59,9.468765,7.778343,98.525757,75,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,349,1.0,shahsaumil@gmail.com,0x0230c0f62a61d77c62c48b162f6dec2be8dc4d20,SAUMIL G SHAH,42,1231488335706197333,0x,NaN,6,...,30,12,6,13,7.260738,11.712182,14.373036,18,1,0
299,350,0.0,b_bordenave@live.com,0x06f62b8b2eebde73dd4f0e1fa9b1321d63fd5c3a,Benjamin Bordenave,42,1827549469105083427,0x,NaN,13,...,29,21,8,9,12.646482,22.980449,56.148393,12,1,2
300,352,0.0,ltmemories@gmail.com,0xd5abc758d753b68985b2bb83f2b2f4030179ea20,John Gravagna,42,-3082015307768915211,0x,NaN,0,...,0,0,0,0,NaN,NaN,NaN,0,0,0
301,353,0.0,seleznyov@gmail.com,0x0cead33eb6adf4860b6b8f66687f51567e111973,ANDRII SELEZNOV,42,7925451324816931321,0x,NaN,2,...,13,5,5,9,5.982765,231.186288,16.922603,11,2,10


In [341]:
df3.to_csv("second_data1709.csv")

## Get EL

In [150]:
df_test
normal_trans_ELy = df_test[["tx_from", "tx_to"]]
normal_trans_ELy["weight"] = df_test["eth_valueUSD"]+df_test["transaction_costUSD"]

<ipython-input-150-5c196cef28d0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_trans_ELy["weight"] = df_test["eth_valueUSD"]+df_test["transaction_costUSD"]


In [151]:
normal_trans_ELy

,tx_from,tx_to,weight
0,0x003e36550908907c2a2da960fd19a419b9a774b7,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,3.231882
1,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,1.871219
2,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,28.790726
3,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0xb1690c08e213a35ed9bab7b318de14420fb57d8c,1.951559
4,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x06012c8cf97bead5deae237070f9587f8e7a266d,6.170015
...,...,...,...
64636,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,209.432872
64637,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xebc86fb12ab0ffac6cbcafce2f049bfe7efada0d,6.761937
64638,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,3.670574
64639,0x2fe4b878e1fd34e7ec449e7521b8333495608805,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,20.030204


In [188]:
EL_ERCneighb_USD.head()

,Unnamed: 0,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,...,cumulativeGasUsed,tokenName,tokenSymbol,tokenDecimal,id,neighboar,Close,eth_valueUSD,transaction_costUSD,will_gas_PriceUSD
0,0,0xeef7c5204ac125cb0975e2a111881d34607f006c9b0d...,0x7769f394f87204df364b219821a068989dc19302,0x4d39217fd94abf03928da97488b16ac91816108f,2.000000e-07,False,True,2018-01-19 11:05:00,1555,3123860.0,...,5767884,ArtisTurba,ARTIS,8.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1045.50,2.091000e-04,79.026940,0.000026
1,1,0x66e267f021407d5d348825f00fab3a508d40ba5bbbcb...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1.000000e+03,False,True,2018-06-20 06:39:00,12,52909.0,...,1924647,Pundi X Token,NPXS,18.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,521.42,5.214200e+05,1.131100,0.000021
2,2,0xa365ef5b0ec87c8b577218d558626ef566adecfc8d55...,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,1.680967e+05,False,True,2018-06-20 06:46:00,13,38101.0,...,1802212,Pundi X Token,NPXS,18.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,520.66,8.752121e+07,0.493138,0.000021
3,891,0xfb30010a991322a51965b3764141ed9ed1d10697910c...,0xdf4943c61042919bd45312d033bd697a1e0a6397,0xa277c67ca954582d6da335b567acd73b02eba6d6,1.000000e+03,False,True,2018-06-20 06:46:00,9,6000000.0,...,5111710,Doves,Dove,18.0,0x387c50f4767b04e7a41f0c365b9d563ae787ec40,0xa277c67ca954582d6da335b567acd73b02eba6d6,520.66,5.206600e+05,5.927642,0.000002
4,3,0x94f95a7eb4ace9415e4e6758c84b8076e8be8547711f...,0x4d39217fd94abf03928da97488b16ac91816108f,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,1.690967e+05,True,True,2018-06-20 09:21:00,2,76202.0,...,1495070,Pundi X Token,NPXS,18.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,0x4d39217fd94abf03928da97488b16ac91816108f,526.87,8.909196e+07,0.243424,0.000011


In [187]:
EL_ERCneighb_USD.head()
finEL_ERCneighb_USD = EL_ERCneibUSD_small[["tx_from", "tx_to"]]#take only 10%
finEL_ERCneighb_USD["weight"] = EL_ERCneibUSD_small["eth_valueUSD"]+EL_ERCneibUSD_small["transaction_costUSD"]


<ipython-input-187-e9849771caee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finEL_ERCneighb_USD["weight"] = EL_ERCneibUSD_small["eth_valueUSD"]+EL_ERCneibUSD_small["transaction_costUSD"]


In [153]:
df_origin_erc20_USD.head()
EL_origin_erc20y = df_origin_erc20_USD[["tx_from", "tx_to"]]
EL_origin_erc20y["weight"] = df_origin_erc20_USD["eth_valueUSD"]+df_origin_erc20_USD["transaction_costUSD"]

<ipython-input-153-3a6d3bbca05f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EL_origin_erc20y["weight"] = df_origin_erc20_USD["eth_valueUSD"]+df_origin_erc20_USD["transaction_costUSD"]


In [189]:
small_Neighb_normal_USD.head()
EL_Neigh_norm = small_Neighb_normal_USD[["tx_from", "tx_to"]]
EL_Neigh_norm["weight"] = small_Neighb_normal_USD["eth_valueUSD"]+small_Neighb_normal_USD["transaction_costUSD"]


<ipython-input-189-098846bb639c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EL_Neigh_norm["weight"] = small_Neighb_normal_USD["eth_valueUSD"]+small_Neighb_normal_USD["transaction_costUSD"]


In [190]:
EL_final = normal_trans_ELy.append([EL_Neigh_norm, EL_origin_erc20y, finEL_ERCneighb_USD], ignore_index = True)

In [193]:
EL_only_origin = normal_trans_ELy.append(EL_origin_erc20y, ignore_index = True)

In [196]:
EL_only_origin.isnull().sum()

tx_from    0
tx_to      0
weight     0
dtype: int64

In [191]:
len(EL_final)

3879723

In [197]:
len(EL_only_origin)

120461

In [200]:
EL_only_origin.to_csv("EL_only_origin.csv")

In [156]:
df_test.loc[df_test["tx_from"]=="0x090dcf87443e1495b0ffaf1f3b7a1d8b2c91aa3b"]

,hash,tx_from,tx_to,eth_value,out,confirmed,timeStamp,nonce,gas,will_gas_Price,gasUsed,transaction_cost,data_transaction,isError,cumulativeGasUsed,Close,eth_valueUSD,transaction_costUSD,will_gas_PriceUSD


In [175]:
import networkx as nx
graph = nx.from_pandas_edgelist(EL_final, 'tx_from', 'tx_to', ['weight'], create_using=nx.DiGraph())

In [198]:
graph_onlyorigin = nx.from_pandas_edgelist(EL_only_origin, 'tx_from', 'tx_to', ['weight'], create_using=nx.DiGraph())

In [199]:
print(nx.info(graph_onlyorigin))

Name: 
Type: DiGraph
Number of nodes: 13909
Number of edges: 33102
Average in degree:   2.3799
Average out degree:   2.3799


In [159]:
EL_final.loc[EL_final["tx_to"]=="0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9"]

,tx_from,tx_to,weight
1267,0x39afa3cddadcea9d91c12968b7963340f5793ce8,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,5.443185e+00
1272,0x2ca53c3ea2fbddda192b8708684d351565e6a540,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,9.294811e+00
1279,0x882d2107790bdb19c93355c4b85e4dfe655a054e,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,1.172887e+01
2618,0xbeaf9ef2c2401352aec6bcdb1274e0a8643d1313,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,2.276290e+00
2748,0xbe6fe69842c1dcb98da4616397f608f795d100b4,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,4.763757e+00
...,...,...,...
19496574,0x16b4096d00a2063005168b8e130c5abfda86cf54,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,1.660059e+06
26119118,0x0751d473f9fb8573e0dff64d75a1da2c67e0069d,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,2.032837e+02
27201263,0x8363aaa98ca3a3636a8f27e48eccd63d0ea9901e,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,3.918620e+00
34453612,0x16b4096d00a2063005168b8e130c5abfda86cf54,0x3d6f0dea3ac3c607b3998e6ce14b6350721752d9,1.660059e+06


In [176]:
 
df3_clean = df3[df3.wallet_address != "0xc93793b1ca80b3b3a3fb698690d4f6e6c684a074"] 
df3_clean = df3_clean.loc[(df3_clean["num_in_trans"]!= 0)|(df3_clean["num_out_trans"]!= 0)] #delete all observations without meaningful transactions as they just lead to many NAs (in the real application one can create another prediction algorithm that works only based on age and location etc.) 

In [174]:
df3_zeros = df3.loc[(df3["num_in_trans"]== 0)|(df3["num_out_trans"]== 0)]#take a look at the zeros -> indeed there is no logic e.g. only millionaires give wrong addresses -> it is always the case
perc_acc_inv = df3_zeros.sow_estimated_net_worth_value.sum() / df3_zeros.sow_estimated_net_worth_value.count()
perc_acc_inv

0.5588235294117647

In [179]:
#calc nodes of every egonet (1hop)
no_nodes_egonet=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address)
    no_nodes_egonet.append(ego_net.order())
df3_clean.loc[:,"no_nodes"]= no_nodes_egonet

KeyboardInterrupt: 

In [ ]:
df3

In [ ]:
#calc nodes of every egonet (2hop)
no_nodes_egonet=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True, radius = 2)
    no_nodes_egonet.append(ego_net.order())
df3_clean.loc[:,"no_nodes_2hop"]= no_nodes_egonet

In [ ]:
#calc edges of every egonet (2hop)
no_edges_egonet=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True, radius = 2)
    no_edges_egonet.append(ego_net.size())
df3_clean.loc[:,"no_edges"]= no_edges_egonet

In [ ]:
#no of transactions in a 1-hop egonet 
no_edges_egonet=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True)
    no_edges_egonet.append(ego_net.size())
df3_clean.loc[:,"no_outg_trans"]= no_edges_egonet

In [ ]:
#calc avg degree of netw
avg_degree_egon=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True, radius = 2)
    result = pd.DataFrame(nx.degree(ego_net))
    avg_degree_egon.append(result.mean())
df3_clean.loc[:,"avg_degree"]= avg_degree_egon

In [ ]:
#calc degree of node in the total network
degree_total=[]
for address in df3_clean.wallet_address:
    degree_total.append(graph.degree(address))
df3_clean.loc[:,"degree_total"]= degree_total

In [ ]:
#sparsity of the 2 hop egonetwork
sparsity_egonet=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True, radius =2)
    no_nodes = ego_net.order()
    max_edges = no_nodes*(no_nodes-1)/2
    no_edges = ego_net.size()
    sparsity = no_edges/(max_edges+1)#plus 1 aufgrund division by zero
    sparsity_egonet.append(sparsity)
df3_clean.loc[:,"sparsity_egonet"]= sparsity_egonet

In [ ]:
#sparsity of the 1 hop egonetwork
sparsity_egonet=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True, radius =1)
    no_nodes = ego_net.order()
    max_edges = no_nodes*(no_nodes-1)/2
    no_edges = ego_net.size()
    sparsity = no_edges/(max_edges+1)#plus 1 aufgrund division by zero
    sparsity_egonet.append(sparsity)
df3_clean.loc[:,"sparsity_egonet"]= sparsity_egonet

In [ ]:
#Degree Centrality of the address in a 2hop ego net
deg_centr_ego=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True, radius = 2)
    x = nx.degree_centrality(ego_net)
    y = pd.DataFrame.from_dict(x, orient = "index")
    z = y.loc[str(address)]
    deg_centr_ego.append(z.iloc[0])
df3_clean.loc[:,"deg_centr_ego"]= deg_centr_ego

In [ ]:
#reciprocity of the address in the ego net
recipr_in_ego=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True)
    x = nx.reciprocity(ego_net, nodes = address)
    recipr_in_ego.append(x)
df3_clean.loc[:,"recipr_in_ego"]= recipr_in_ego

In [ ]:
#reciprocity of the address in the ego net
recipr_in_ego=[]
for address in df3_clean.wallet_address:
    ego_net = nx.ego_graph(graph, address, undirected=True)
    x = nx.reciprocity(ego_net, nodes = address)
    recipr_in_ego.append(x)
df3_clean.loc[:,"recipr_in_ego"]= recipr_in_ego

In [ ]:
df3_clean

In [582]:
x = nx.degree_centrality(ego_net)
y= pd.DataFrame.from_dict(x, orient = "index")
z = y.loc["0xa886576fed10723401386697aadf777e1d4bca05"]
z.iloc[0]

0.03754266211604096

In [385]:
# average percentage of transaction costs comp. to transferred ETH in outgoing transactions
rel_trans_cost=[]
for address in df3.wallet_address:
    filtered_df = df_test.loc[(df_test['tx_from'] == address)]
    result_p_trans=[]
    for index, row in filtered_df.iterrows():
        result = row.transaction_costUSD/row.eth_valueUSD
        result_p_trans.append(result)
    rel_trans_cost = rel_trans_cost.append(result.mean())
    
df3.loc[:,"rel_trans_cost"]= rel_trans_cost

Index(['sow_estimated_net_worth_value', 'address', 'wallet_address',
       'full_name', 'address_length', 'hash', 'address_beginning',
       'num_in_trans_1neighb', 'avg_tx_out_neighb', 'num_out_trans_1neighb',
       'avg_tx_in_neighb', 'max_tx_out_neighb', 'max_tx_in_neighb',
       'sum_tx_in_neighb', 'sum_tx_out_neighb', 'std_tx_out_neighb',
       'std_tx_in_neighb', 'unique_tx_out_neighb', 'unique_tx_in_neighb',
       'avg_txcost_out_neighb', 'avg_txcost_in_neighb', 'max_txcost_in_neighb',
       'max_txcost_out_neighb', 'std_txcost_out_neighb',
       'std_txcost_in_neighb', 'address_balance', 'timedelta_trans_out',
       'num_in_trans', 'num_out_trans', 'avg_trans_out', 'avg_trans_in',
       'sum_trans_in', 'sum_trans_out', 'max_trans_out', 'max_trans_in',
       'std_trans_in', 'std_trans_out', 'min_trans_out', 'min_trans_in',
       'unique_tpartn_out', 'unique_tpartn_in', 'avg_transcost_out',
       'avg_transcost_in', 'max_transcost_out', 'max_transcost_in',
       'st

In [256]:
#std amount ingoing transaction cost
std_txcost_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].std())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    std_txcost_in_neighb.append(result)
        
df3.loc[:,"std_txcost_in_neighb"]= std_txcost_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
